In [1]:
#5413 paramètres transférés dans le cas usuel
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import copy
import os
import time
import sys

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def lstm_cell_forward(xt, a_prev, c_prev, parameters):
    """
    Arguments:
    xt -- Données d'entrée à time-step t, array de forme (n_x, m)
    a_prev -- Etat caché précédent, array de forme (n_a, m)
    c_prev -- Etat de la cellule précédente, array de forme (n_a, m)
    parameters -- Dictionnaire Python contenant:
                Wf -- Poids de la forget gate, array de forme (n_a, n_a + n_x)
                bf -- Biais de la forget gate, array de forme (n_a, 1)
                Wi -- Poids de l'update gate, array de forme (n_a, n_a + n_x)
                bi -- Biais de l'update gate, array de forme (n_a, 1)
                Wc -- Poids de la première "tanh", array de forme (n_a, n_a + n_x)
                bc -- Biais de la première "tanh", array de forme (n_a, 1)
                Wo -- Poids de l'output gate, array de forme (n_a, n_a + n_x)
                bo -- Biais de l'output gate, array de forme (n_a, 1)
                Wy -- Poids pour l'état caché, array de forme (n_y, n_a)
                by -- Biais pour l'état caché, array de forme (n_y, 1)

    Returns:
    a_next -- Prochain état caché, array de forme (n_a, m)
    c_next -- Prochain état de cellule, array de forme (n_a, m)
    yt_pred -- Prédiction à time-step t, array de forme (n_y, m)
    cache -- Tuple de valeurs pour la backpropagation
    """

    # Récupérer les paramètres du dictionnaire
    Wf = parameters["Wf"]
    bf = parameters["bf"]
    Wi = parameters["Wi"]
    bi = parameters["bi"]
    Wc = parameters["Wc"]
    bc = parameters["bc"]
    Wo = parameters["Wo"]
    bo = parameters["bo"]
    Wy = parameters["Wy"]
    by = parameters["by"]

    # Récupérer les dimensions
    n_x, m = xt.shape
    n_y, n_a = Wy.shape

    # Concaténer a_prev et xt
    concat = np.zeros((n_a + n_x, m))
    concat[: n_a, :] = a_prev
    concat[n_a:, :] = xt

    # Calculer les valeurs pour ft, it, cct, c_next, ot, a_next
    ft = sigmoid(np.matmul(Wf, concat) + bf)
    it = sigmoid(np.matmul(Wi, concat) + bi)
    cct = np.tanh(np.matmul(Wc, concat) + bc)
    c_next = (ft * c_prev) + (it * cct)
    ot = sigmoid(np.matmul(Wo, concat) + bo)
    a_next = ot * np.tanh(c_next)

    # Calculer la prédiction
    yt_pred = softmax(np.matmul(Wy, a_next) + by)

    # Stocker les valeurs pour la backpropagation
    cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)

    return a_next, c_next, yt_pred, cache

def lstm_cell_backward(da_next, dc_next, cache):
    """
    Arguments:
    da_next -- Gradient du prochain état caché, array de forme (n_a, m)
    dc_next -- Gradient du prochain état de cellule, array de forme (n_a, m)
    cache -- Cache du forward pass

    Returns:
    gradients -- Dictionnaire contenant les gradients
    """

    # Récupérer les informations du cache
    (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters) = cache

    # Récupérer les dimensions
    n_x, m = xt.shape
    n_a, m = a_next.shape

    # Calculer les dérivées des portes
    dot = da_next * np.tanh(c_next) * ot * (1 - ot)
    dcct = (dc_next * it + ot * (1 - np.square(np.tanh(c_next))) * it * da_next) * (1 - np.square(cct))
    dit = (dc_next * cct + ot * (1 - np.square(np.tanh(c_next))) * cct * da_next) * it * (1 - it)
    dft = (dc_next * c_prev + ot * (1 - np.square(np.tanh(c_next))) * c_prev * da_next) * ft * (1 - ft)

    concat = np.concatenate((a_prev, xt), axis=0)

    # Calculer les dérivées des paramètres
    dWf = np.dot(dft, concat.T)
    dWi = np.dot(dit, concat.T)
    dWc = np.dot(dcct, concat.T)
    dWo = np.dot(dot, concat.T)
    dbf = np.sum(dft, axis=1, keepdims=True)
    dbi = np.sum(dit, axis=1, keepdims=True)
    dbc = np.sum(dcct, axis=1, keepdims=True)
    dbo = np.sum(dot, axis=1, keepdims=True)

    # Calculer les dérivées par rapport à l'état caché précédent, l'état de cellule précédent et l'entrée
    da_prev = np.dot(parameters['Wf'][:, :n_a].T, dft) + np.dot(parameters['Wi'][:, :n_a].T, dit) + np.dot(
        parameters['Wc'][:, :n_a].T, dcct) + np.dot(parameters['Wo'][:, :n_a].T, dot)
    dc_prev = dc_next * ft + ot * (1 - np.square(np.tanh(c_next))) * ft * da_next
    dxt = np.dot(parameters['Wf'][:, n_a:].T, dft) + np.dot(parameters['Wi'][:, n_a:].T, dit) + np.dot(
        parameters['Wc'][:, n_a:].T, dcct) + np.dot(parameters['Wo'][:, n_a:].T, dot)

    # Sauvegarder les gradients
    gradients = {"dxt": dxt, "da_prev": da_prev, "dc_prev": dc_prev, "dWf": dWf, "dbf": dbf, "dWi": dWi, "dbi": dbi,
                 "dWc": dWc, "dbc": dbc, "dWo": dWo, "dbo": dbo}

    return gradients

def lstm_forward(x, a0, parameters):
    """
    Arguments:
    x -- Données d'entrée pour chaque time-step, array de forme (n_x, m, T_x)
    a0 -- État caché initial, array de forme (n_a, m)
    parameters -- Dictionnaire Python des paramètres LSTM

    Returns:
    a -- États cachés pour chaque time-step, array de forme (n_a, m, T_x)
    y -- Prédictions pour chaque time-step, array de forme (n_y, m, T_x)
    c -- États de cellule pour chaque time-step, array de forme (n_a, m, T_x)
    caches -- Tuple de valeurs pour la backpropagation
    """

    # Initialiser les caches
    caches = []

    # Récupérer les dimensions
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wy"].shape

    # Initialiser a, c et y avec des zéros
    a = np.zeros((n_a, m, T_x))
    c = a.copy()
    y = np.zeros((n_y, m, T_x))

    # Initialiser a_next et c_next
    a_next = a0
    c_next = np.zeros(a_next.shape)

    # Boucle sur tous les time-steps
    for t in range(T_x):
        # Mettre à jour a_next, c_next, calculer la prédiction et obtenir le cache
        a_next, c_next, yt, cache = lstm_cell_forward(x[:, :, t], a_next, c_next, parameters)
        # Sauvegarder les valeurs
        a[:, :, t] = a_next
        y[:, :, t] = yt
        c[:, :, t] = c_next
        # Ajouter le cache
        caches.append(cache)

    # Stocker les valeurs pour la backpropagation
    caches = (caches, x)

    return a, y, c, caches

def lstm_backward(da, caches):
    """
    Arguments:
    da -- Gradient par rapport aux états cachés, array de forme (n_a, m, T_x)
    caches -- Cache du forward pass

    Returns:
    gradients -- Dictionnaire Python contenant les gradients
    """

    # Récupérer les valeurs du cache
    (caches, x) = caches
    (a1, c1, a0, c0, f1, i1, cc1, o1, x1, parameters) = caches[0]

    # Récupérer les dimensions
    n_a, m, T_x = da.shape
    n_x, m = x1.shape

    # Initialiser les gradients
    dx = np.zeros((n_x, m, T_x))
    da0 = np.zeros((n_a, m))
    da_prevt = np.zeros(da0.shape)
    dc_prevt = np.zeros(da0.shape)
    dWf = np.zeros((n_a, n_a + n_x))
    dWi = np.zeros(dWf.shape)
    dWc = np.zeros(dWf.shape)
    dWo = np.zeros(dWf.shape)
    dbf = np.zeros((n_a, 1))
    dbi = np.zeros(dbf.shape)
    dbc = np.zeros(dbf.shape)
    dbo = np.zeros(dbf.shape)

    # Boucle sur la séquence en sens inverse
    for t in reversed(range(T_x)):
        # Calculer tous les gradients à l'aide de lstm_cell_backward
        gradients = lstm_cell_backward(da[:, :, t] + da_prevt, dc_prevt, caches[t])
        # Stocker ou ajouter les gradients aux gradients de l'étape précédente
        dx[:, :, t] = gradients["dxt"]
        dWf += gradients["dWf"]
        dWi += gradients["dWi"]
        dWc += gradients["dWc"]
        dWo += gradients["dWo"]
        dbf += gradients["dbf"]
        dbi += gradients["dbi"]
        dbc += gradients["dbc"]
        dbo += gradients["dbo"]
        da_prevt = gradients["da_prev"]
        dc_prevt = gradients["dc_prev"]

    # Définir le premier gradient d'activation
    da0 = gradients["da_prev"]

    # Stocker les gradients dans un dictionnaire Python
    gradients = {"dx": dx, "da0": da0, "dWf": dWf, "dbf": dbf, "dWi": dWi, "dbi": dbi,
                 "dWc": dWc, "dbc": dbc, "dWo": dWo, "dbo": dbo}

    return gradients

def initialize_adam_for_lstm(parameters):
    """
    Initialise v et s pour les paramètres du LSTM.

    Arguments:
    parameters -- Dictionnaire Python contenant les paramètres du LSTM.

    Returns:
    v -- Dictionnaire Python qui contiendra la moyenne mobile exponentielle du gradient.
    s -- Dictionnaire Python qui contiendra la moyenne mobile exponentielle du carré du gradient.
    """
    v = {}
    s = {}

    # Initialiser v, s pour tous les paramètres du LSTM
    for key in parameters.keys():
        v["d" + key] = np.zeros_like(parameters[key])
        s["d" + key] = np.zeros_like(parameters[key])

    return v, s

def update_parameters_with_adam_for_lstm(parameters, grads, v, s, t, learning_rate=0.01,
                                         beta1=0.9, beta2=0.999, epsilon=1e-8):
    """
    Update parameters using Adam

    Arguments:
    parameters -- dictionary containing your parameters
    grads -- dictionary containing your gradients, output of lstm_backward
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    t -- Timestep, integer
    learning_rate -- the learning rate, scalar
    beta1 -- Exponential decay hyperparameter for the first moment estimates
    beta2 -- Exponential decay hyperparameter for the second moment estimates
    epsilon -- hyperparameter preventing division by zero in Adam updates

    Returns:
    parameters -- python dictionary containing your updated parameters
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    """
    v_corrected = {}  # Estimation du premier moment corrigée du biais
    s_corrected = {}  # Estimation du second moment corrigée du biais

    # Effectuer la mise à jour Adam sur tous les paramètres
    for key in parameters.keys():
        # Clé correspondante dans les dictionnaires grads, v, s
        d_key = "d" + key

        # S'assurer que nous avons le gradient correspondant
        if d_key not in grads:
            continue

        # Moyenne mobile des gradients
        v[d_key] = beta1 * v[d_key] + (1 - beta1) * grads[d_key]

        # Calcul de l'estimation du premier moment corrigée du biais
        v_corrected[d_key] = v[d_key] / (1 - beta1**t)

        # Moyenne mobile des carrés des gradients
        s[d_key] = beta2 * s[d_key] + (1 - beta2) * (grads[d_key]**2)

        # Calcul de l'estimation du second moment corrigée du biais
        s_corrected[d_key] = s[d_key] / (1 - beta2**t)

        # Mise à jour des paramètres
        parameters[key] = parameters[key] - learning_rate * v_corrected[d_key] / (np.sqrt(s_corrected[d_key]) + epsilon)

    return parameters, v, s

def initialize_lstm_parameters(n_a, n_x, n_y):
    """
    Initialise les paramètres du LSTM.

    Arguments:
    n_a -- nombre d'unités dans la couche cachée
    n_x -- taille d'entrée
    n_y -- taille de sortie

    Returns:
    parameters -- dictionnaire Python contenant les paramètres initialisés
    """
    np.random.seed(1)

    # Initialisation avec He/Xavier
    Wf = np.random.randn(n_a, n_a + n_x) * np.sqrt(1. / (n_a + n_x))
    bf = np.zeros((n_a, 1))
    Wi = np.random.randn(n_a, n_a + n_x) * np.sqrt(1. / (n_a + n_x))
    bi = np.zeros((n_a, 1))
    Wc = np.random.randn(n_a, n_a + n_x) * np.sqrt(1. / (n_a + n_x))
    bc = np.zeros((n_a, 1))
    Wo = np.random.randn(n_a, n_a + n_x) * np.sqrt(1. / (n_a + n_x))
    bo = np.zeros((n_a, 1))
    Wy = np.random.randn(n_y, n_a) * np.sqrt(1. / n_a)
    by = np.zeros((n_y, 1))

    parameters = {"Wf": Wf, "bf": bf, "Wi": Wi, "bi": bi, "Wc": Wc, "bc": bc, "Wo": Wo, "bo": bo, "Wy": Wy, "by": by}

    return parameters

def train_lstm(X_train, Y_train, n_a, n_x, n_y, num_epochs=10, seed=1, learning_rate=0.01, initial_params=None):
    """
    Entraîne un LSTM sur les données fournies, avec possibilité d'initialiser avec des paramètres existants.

    Arguments:
    X_train -- données d'entrée, numpy array de forme (n_x, m, T_x)
    Y_train -- étiquettes, numpy array de forme (n_y, m, T_x)
    n_a -- nombre d'unités dans la couche cachée
    n_x -- taille d'entrée
    n_y -- taille de sortie
    num_epochs -- nombre d'époques d'entraînement
    seed -- graine pour la reproductibilité
    learning_rate -- taux d'apprentissage
    initial_params -- paramètres initiaux (optionnel)

    Returns:
    parameters -- paramètres finaux
    parameters_history -- historique des paramètres à chaque époque
    loss_history -- historique des pertes
    """
    np.random.seed(seed)

    # Initialisation des paramètres
    if initial_params is None:
        parameters = initialize_lstm_parameters(n_a, n_x, n_y)
    else:
        parameters = copy.deepcopy(initial_params)

    parameters_history = []
    loss_history = []

    # Initialiser Adam
    v, s = initialize_adam_for_lstm(parameters)
    t = 0  # Compteur pour Adam

    for epoch in range(num_epochs):
        print(f"Époque {epoch+1}/{num_epochs}")

        # Forward pass
        a0 = np.zeros((n_a, X_train.shape[1]))
        a, y_pred, c, caches = lstm_forward(X_train, a0, parameters)

        # Calcul de la perte (cross-entropy)
        loss = -np.sum(Y_train * np.log(y_pred + 1e-8)) / (Y_train.shape[1] * Y_train.shape[2])
        loss_history.append(loss)
        print(f"Loss: {loss:.4f}")

        # Initialisation du gradient de sortie
        da = np.zeros_like(a)

        # Créer un dictionnaire complet pour les gradients
        gradients = {}

        # Pour chaque pas de temps, calculer le gradient
        dWy = np.zeros_like(parameters["Wy"])
        dby = np.zeros_like(parameters["by"])

        for t_idx in range(Y_train.shape[2]):
            # Gradient de la cross-entropy
            dy = y_pred[:, :, t_idx] - Y_train[:, :, t_idx]
            # Accumuler les gradients pour Wy et by
            dWy += np.dot(dy, a[:, :, t_idx].T)
            dby += np.sum(dy, axis=1, keepdims=True)
            # Gradient par rapport à a
            da[:, :, t_idx] = np.dot(parameters["Wy"].T, dy)

        # Backward pass pour le reste des paramètres LSTM
        lstm_gradients = lstm_backward(da, caches)

        # Combiner tous les gradients
        gradients = lstm_gradients.copy()
        gradients["dWy"] = dWy
        gradients["dby"] = dby

        # Mise à jour des paramètres avec Adam
        t += 1
        parameters, v, s = update_parameters_with_adam_for_lstm(parameters, gradients, v, s, t, learning_rate)

        # Sauvegarde des paramètres après cette époque
        parameters_history.append(copy.deepcopy(parameters))

    return parameters, parameters_history, loss_history

def evaluate_lstm(X_test, Y_test, parameters):
    """
    Évalue les performances d'un LSTM pour la régression.

    Arguments:
    X_test -- données de test, numpy array de forme (n_x, m, T_x)
    Y_test -- étiquettes de test, numpy array de forme (n_y, m, T_x)
    parameters -- dictionnaire Python contenant les paramètres du LSTM

    Returns:
    rmse_val -- erreur quadratique moyenne
    loss -- perte du modèle
    """
    n_a = parameters["Wf"].shape[0]

    # Forward pass
    a0 = np.zeros((n_a, X_test.shape[1]))
    _, y_pred, _, _ = lstm_forward(X_test, a0, parameters)

    # Calculer RMSE
    rmse_val = np.sqrt(np.mean((y_pred - Y_test) ** 2))

    # Calcul de la perte (MSE)
    loss = np.mean((y_pred - Y_test) ** 2)

    return rmse_val, loss

def rmse(predictions, targets):
    """
    Calcule la racine de l'erreur quadratique moyenne (RMSE)

    Arguments:
    predictions -- prédictions du modèle, array numpy
    targets -- valeurs cibles, array numpy

    Returns:
    rmse_value -- valeur RMSE (float)
    """
    return np.sqrt(np.mean((predictions - targets) ** 2))

def flatten_parameters(parameters):
    """
    Aplatit les paramètres d'un LSTM en un seul vecteur.

    Arguments:
    parameters -- dictionnaire Python contenant les paramètres

    Returns:
    flattened -- vecteur aplati des paramètres
    param_shapes -- formes originales des paramètres
    param_sizes -- tailles des paramètres aplatis
    """
    flattened = []
    param_shapes = {}
    param_sizes = {}

    for key in ["Wf", "bf", "Wi", "bi", "Wc", "bc", "Wo", "bo", "Wy", "by"]:
        param = parameters[key]
        param_shapes[key] = param.shape
        flattened.append(param.flatten())
        param_sizes[key] = param.size

    return np.concatenate(flattened), param_shapes, param_sizes

def unflatten_parameters(flattened, param_shapes, param_sizes):
    """
    Restaure un vecteur aplati de paramètres à leur forme originale.

    Arguments:
    flattened -- vecteur aplati des paramètres
    param_shapes -- formes originales des paramètres
    param_sizes -- tailles des paramètres aplatis

    Returns:
    parameters -- dictionnaire Python contenant les paramètres
    """
    parameters = {}
    start = 0

    for key in ["Wf", "bf", "Wi", "bi", "Wc", "bc", "Wo", "bo", "Wy", "by"]:
        size = param_sizes[key]
        parameters[key] = flattened[start:start+size].reshape(param_shapes[key])
        start += size

    return parameters



def calculate_transmission_size(parameters=None, n_clusters=None, transition_matrix=None, client_centers=None):
    """
    Calcule la taille de transmission en octets.

    Args:
        parameters: Dictionnaire ou liste des paramètres originaux (arrays numpy)
        n_clusters: Nombre de clusters utilisés
        transition_matrix: Matrice de transition entre clusters
        client_centers: Centres des clusters par client (liste de tuples (centers, transition_matrix))

    Returns:
        Taille totale en octets
    """
    total_size = 0

    # Cas 1: Paramètres originaux (dictionnaire)
    if parameters is not None and isinstance(parameters, dict):
        for key in parameters:
            if hasattr(parameters[key], 'size'):
                # Chaque nombre flottant occupe 4 octets (float32)
                total_size += parameters[key].size * 4
        return total_size

    # Cas 2: Paramètres originaux (liste)
    elif parameters is not None and isinstance(parameters, list) and all(hasattr(p, 'size') for p in parameters if p is not None):
        for param in parameters:
            if param is not None and hasattr(param, 'size'):
                # Chaque nombre flottant occupe 4 octets (float32)
                total_size += param.size * 4
        return total_size

    # Cas 3: Clusterisation globale avec n_clusters et transition_matrix
    elif n_clusters is not None and transition_matrix is not None:
        # Taille de la matrice de transition
        if hasattr(transition_matrix, 'size'):
            total_size += transition_matrix.size * 4

        # Si nous avons des centres de clusters explicites
        if client_centers is not None and not isinstance(client_centers, list):
            if hasattr(client_centers, 'size'):
                total_size += client_centers.size * 4

        return total_size

    # Cas 4: Clusterisation locale avec client_centers
    elif client_centers is not None and isinstance(client_centers, list):
        for client_data in client_centers:
            # Dans la clusterisation locale, client_data est un tuple (cluster_centers, transition_matrix)
            if isinstance(client_data, tuple) and len(client_data) == 2:
                centers, trans_matrix = client_data

                # Taille des centres de clusters
                if hasattr(centers, 'size'):
                    total_size += centers.size * 4

                # Taille de la matrice de transition
                if hasattr(trans_matrix, 'size'):
                    total_size += trans_matrix.size * 4

        return total_size

    # En cas d'erreur ou de paramètres non valides
    raise ValueError("Entrée invalide pour le calcul de la taille de transmission")


def cluster_parameters_by_epoch_local(parameters_history_by_seed, n_clusters=3):
    """
    Clusterise les paramètres à chaque époque pour chaque seed.
    Cette version est utilisée pour la clusterisation locale par client.

    Arguments:
    parameters_history_by_seed -- liste de listes de dictionnaires Python contenant les paramètres
    n_clusters -- nombre de clusters à former

    Returns:
    kmeans_models -- modèles KMeans pour chaque époque
    cluster_labels -- étiquettes de cluster pour chaque graine à chaque époque
    flat_params -- paramètres aplatis pour chaque graine à chaque époque
    param_shapes -- formes des paramètres
    param_sizes -- tailles des paramètres aplatis
    """
    n_seeds = len(parameters_history_by_seed)
    n_epochs = len(parameters_history_by_seed[0])

    kmeans_models = []
    cluster_labels = np.zeros((n_seeds, n_epochs), dtype=int)
    flat_params = []

    # Obtenir les formes et tailles des paramètres
    _, param_shapes, param_sizes = flatten_parameters(parameters_history_by_seed[0][0])

    # Aplatir les paramètres pour toutes les graines et époques
    for seed in range(n_seeds):
        seed_params = []
        for epoch in range(n_epochs):
            flattened, _, _ = flatten_parameters(parameters_history_by_seed[seed][epoch])
            seed_params.append(flattened)
        flat_params.append(seed_params)

    flat_params = np.array(flat_params)

    # Clusteriser par époque
    for epoch in range(n_epochs):
        epoch_params = flat_params[:, epoch, :]
        kmeans = KMeans(n_clusters=min(n_clusters, n_seeds), random_state=42)
        cluster_labels[:, epoch] = kmeans.fit_predict(epoch_params)
        kmeans_models.append(kmeans)

    return kmeans_models, cluster_labels, flat_params, param_shapes, param_sizes

def compute_transition_matrix_local(cluster_labels):
    """
    Calcule la matrice de transition de Markov à partir des séquences de labels de clusters.
    Cette version est utilisée pour la clusterisation locale par client.

    Arguments:
    cluster_labels -- étiquettes de cluster pour chaque graine à chaque époque

    Returns:
    transition_matrix -- matrice de transition de Markov
    """
    n_seeds, n_epochs = cluster_labels.shape
    n_clusters = np.max(cluster_labels) + 1

    transition_counts = np.zeros((n_clusters, n_clusters))

    # Compter les transitions
    for seed in range(n_seeds):
        for epoch in range(n_epochs - 1):
            from_cluster = cluster_labels[seed, epoch]
            to_cluster = cluster_labels[seed, epoch + 1]
            transition_counts[from_cluster, to_cluster] += 1

    # Normaliser pour obtenir les probabilités
    transition_matrix = np.zeros_like(transition_counts)
    for i in range(n_clusters):
        row_sum = np.sum(transition_counts[i])
        if row_sum > 0:
            transition_matrix[i] = transition_counts[i] / row_sum
        else:
            # Si aucune transition n'est observée depuis ce cluster, distribution uniforme
            transition_matrix[i] = 1.0 / n_clusters

    return transition_matrix

def simulate_parameter_trajectory_local(initial_cluster, transition_matrix, n_steps, kmeans_models):
    """
    Simule une trajectoire de paramètres basée sur la matrice de transition.
    Cette version est utilisée pour la clusterisation locale par client.

    Arguments:
    initial_cluster -- cluster initial
    transition_matrix -- matrice de transition de Markov
    n_steps -- nombre d'étapes à simuler
    kmeans_models -- modèles KMeans pour chaque époque

    Returns:
    trajectory -- trajectoire simulée de paramètres
    cluster_sequence -- séquence de clusters visitée
    """
    n_clusters = transition_matrix.shape[0]
    cluster_sequence = [initial_cluster]
    current_cluster = initial_cluster

    for _ in range(n_steps - 1):
        # Échantillonner le prochain cluster
        next_cluster = np.random.choice(n_clusters, p=transition_matrix[current_cluster])
        cluster_sequence.append(next_cluster)
        current_cluster = next_cluster

    # Convertir la séquence de clusters en paramètres
    trajectory = []
    for step, cluster in enumerate(cluster_sequence):
        # Utiliser le centre du cluster comme paramètres représentatifs
        # Si nous avons dépassé le nombre d'époques dans kmeans_models, utiliser le dernier
        model_idx = min(step, len(kmeans_models) - 1)
        trajectory.append(kmeans_models[model_idx].cluster_centers_[cluster])

    return trajectory, cluster_sequence

def cluster_local_by_client(parameters_history_by_client_seed, n_clusters=3, n_steps=None):
    """
    Effectue une clusterisation locale pour chaque client

    Arguments:
    parameters_history_by_client_seed -- dictionnaire contenant les historiques de paramètres pour chaque client et chaque seed
    n_clusters -- nombre de clusters à former pour chaque client
    n_steps -- nombre d'étapes à simuler (par défaut: même que la longueur d'origine)

    Returns:
    client_clusters -- liste des centres de clusters et matrices de transition pour chaque client
    param_shapes -- formes des paramètres
    param_sizes -- tailles des paramètres
    """
    n_clients = len(parameters_history_by_client_seed)
    client_clusters = []

    # Pour stocker les formes et tailles des paramètres (identiques pour tous les clients)
    param_shapes = None
    param_sizes = None

    for client_id in range(n_clients):
        client_params_history = parameters_history_by_client_seed[client_id]
        n_seeds = len(client_params_history)
        n_epochs = len(client_params_history[0])

        if n_steps is None:
            n_steps = n_epochs

        # Clusteriser les paramètres locaux du client
        kmeans_models, cluster_labels, flat_params, param_shapes, param_sizes = cluster_parameters_by_epoch_local(
            client_params_history, n_clusters=min(n_clusters, n_seeds))

        # Calculer la matrice de transition locale
        transition_matrix = compute_transition_matrix_local(cluster_labels)

        # Extraire les centres de clusters (utiliser le dernier modèle KMeans)
        last_kmeans = kmeans_models[-1]
        cluster_centers = last_kmeans.cluster_centers_

        # Stocker les centres et la matrice pour ce client
        client_clusters.append((cluster_centers, transition_matrix))

    return client_clusters, param_shapes, param_sizes


def cluster_parameters_by_epoch(parameters_history_by_seed, n_clusters=3):
    """
    Clusterise les paramètres à chaque époque pour tous les clients.

    Arguments:
    parameters_history_by_seed -- liste de listes de dictionnaires Python contenant les paramètres
    n_clusters -- nombre de clusters à former

    Returns:
    kmeans_models -- modèles KMeans pour chaque époque
    cluster_labels -- étiquettes de cluster pour chaque graine à chaque époque
    flat_params -- paramètres aplatis pour chaque graine à chaque époque
    param_shapes -- formes des paramètres
    param_sizes -- tailles des paramètres aplatis
    """
    n_seeds = len(parameters_history_by_seed)
    n_epochs = len(parameters_history_by_seed[0])

    kmeans_models = []
    cluster_labels = np.zeros((n_seeds, n_epochs), dtype=int)
    flat_params = []

    # Obtenir les formes et tailles des paramètres
    _, param_shapes, param_sizes = flatten_parameters(parameters_history_by_seed[0][0])

    # Aplatir les paramètres pour toutes les graines et époques
    for seed in range(n_seeds):
        seed_params = []
        for epoch in range(n_epochs):
            flattened, _, _ = flatten_parameters(parameters_history_by_seed[seed][epoch])
            seed_params.append(flattened)
        flat_params.append(seed_params)

    flat_params = np.array(flat_params)

    # Clusteriser par époque
    for epoch in range(n_epochs):
        epoch_params = flat_params[:, epoch, :]
        kmeans = KMeans(n_clusters=min(n_clusters, n_seeds), random_state=42)
        cluster_labels[:, epoch] = kmeans.fit_predict(epoch_params)
        kmeans_models.append(kmeans)

    return kmeans_models, cluster_labels, flat_params, param_shapes, param_sizes

def compute_transition_matrix(cluster_labels):
    """
    Calcule la matrice de transition de Markov à partir des séquences de labels de clusters.

    Arguments:
    cluster_labels -- étiquettes de cluster pour chaque graine à chaque époque

    Returns:
    transition_matrix -- matrice de transition de Markov
    """
    n_seeds, n_epochs = cluster_labels.shape
    n_clusters = np.max(cluster_labels) + 1

    transition_counts = np.zeros((n_clusters, n_clusters))

    # Compter les transitions
    for seed in range(n_seeds):
        for epoch in range(n_epochs - 1):
            from_cluster = cluster_labels[seed, epoch]
            to_cluster = cluster_labels[seed, epoch + 1]
            transition_counts[from_cluster, to_cluster] += 1

    # Normaliser pour obtenir les probabilités
    transition_matrix = np.zeros_like(transition_counts)
    for i in range(n_clusters):
        row_sum = np.sum(transition_counts[i])
        if row_sum > 0:
            transition_matrix[i] = transition_counts[i] / row_sum
        else:
            # Si aucune transition n'est observée depuis ce cluster, distribution uniforme
            transition_matrix[i] = 1.0 / n_clusters

    return transition_matrix

def simulate_parameter_trajectory(initial_cluster, transition_matrix, n_steps, kmeans_models):
    """
    Simule une trajectoire de paramètres basée sur la matrice de transition.

    Arguments:
    initial_cluster -- cluster initial
    transition_matrix -- matrice de transition de Markov
    n_steps -- nombre d'étapes à simuler
    kmeans_models -- modèles KMeans pour chaque époque

    Returns:
    trajectory -- trajectoire simulée de paramètres
    cluster_sequence -- séquence de clusters visitée
    """
    n_clusters = transition_matrix.shape[0]
    cluster_sequence = [initial_cluster]
    current_cluster = initial_cluster

    for _ in range(n_steps - 1):
        # Échantillonner le prochain cluster
        next_cluster = np.random.choice(n_clusters, p=transition_matrix[current_cluster])
        cluster_sequence.append(next_cluster)
        current_cluster = next_cluster

    # Convertir la séquence de clusters en paramètres
    trajectory = []
    for step, cluster in enumerate(cluster_sequence):
        # Utiliser le centre du cluster comme paramètres représentatifs
        # Si nous avons dépassé le nombre d'époques dans kmeans_models, utiliser le dernier
        model_idx = min(step, len(kmeans_models) - 1)
        trajectory.append(kmeans_models[model_idx].cluster_centers_[cluster])

    return trajectory, cluster_sequence

def simulate_full_parameter_transmission(parameters_history_by_client, X_test, Y_test, n_a, n_x, n_y, num_transfer_epochs, learning_rate):
    """
    Simule la méthode traditionnelle de transmission complète des paramètres.

    Arguments:
    parameters_history_by_client -- historique des paramètres pour chaque client
    X_test -- données de test
    Y_test -- étiquettes de test
    n_a -- nombre d'unités cachées
    n_x -- dimension d'entrée
    n_y -- dimension de sortie
    num_transfer_epochs -- nombre d'époques pour le transfer learning
    learning_rate -- taux d'apprentissage

    Returns:
    full_params_avg -- paramètres moyennés sans transfer
    full_params_transfer -- paramètres après transfer
    full_params_acc -- précision après transfer
    full_params_loss -- perte après transfer
    full_params_size -- taille de transmission (en octets)
    """
    # Extraire les derniers paramètres pour chaque client
    last_params = [client_history[-1] for client_history in parameters_history_by_client]

    # Calculer la moyenne des paramètres (FedAvg)
    full_params_avg = {}
    for key in last_params[0].keys():
        full_params_avg[key] = np.mean([params[key] for params in last_params], axis=0)

    # Calculer la taille de transmission
    full_params_size = calculate_transmission_size(full_params_avg)

    # Réentraîner avec transfer learning
    full_params_transfer, loss_history, acc_history = transfer_learning(
        X_train=X_test, Y_train=Y_test,
        X_test=X_test, Y_test=Y_test,
        source_parameters=full_params_avg,
        n_a=n_a, n_x=n_x, n_y=n_y,
        num_epochs=num_transfer_epochs,
        learning_rate=learning_rate
    )

    # Récupérer les performances finales
    full_params_acc = acc_history[-1]
    full_params_loss = loss_history[-1]

    return full_params_avg, full_params_transfer, full_params_acc, full_params_loss, full_params_size


def aggregate_client_clusters(client_clusters, param_shapes, param_sizes, n_clients, weight_by_client=None):
    """
    Agrège les clusters de paramètres de plusieurs clients

    Arguments:
    client_clusters -- liste des centres de clusters et matrices de transition pour chaque client
    param_shapes -- formes des paramètres
    param_sizes -- tailles des paramètres
    n_clients -- nombre de clients
    weight_by_client -- poids pour l'agrégation de chaque client (optionnel)

    Returns:
    aggregated_params -- paramètres agrégés à partir des clusters des clients
    """
    # Si les poids ne sont pas fournis, utiliser un poids uniforme
    if weight_by_client is None:
        weight_by_client = np.ones(n_clients) / n_clients

    # Initialiser les paramètres agrégés
    all_flattened_params = []

    # Pour chaque client, simuler une trajectoire à partir de ses clusters
    for client_id, (cluster_centers, transition_matrix) in enumerate(client_clusters):
        # Choisir un cluster de départ (utiliser le cluster le plus fréquent ou le premier)
        initial_cluster = 0

        # Obtenir un ensemble de paramètres représentatifs de ce client
        # Simplement moyenne des centres de clusters pondérée par la probabilité stationnaire
        n_clusters = transition_matrix.shape[0]

        # Calculer la distribution stationnaire (au lieu de simuler une trajectoire)
        # Méthode simple: itérer la matrice de transition jusqu'à convergence
        pi = np.ones(n_clusters) / n_clusters  # Distribution initiale uniforme
        for _ in range(100):  # Nombre d'itérations arbitraire, devrait converger rapidement
            pi_new = np.dot(pi, transition_matrix)
            if np.allclose(pi, pi_new):
                break
            pi = pi_new

        # Pondérer les centres par la distribution stationnaire
        client_params = np.zeros_like(cluster_centers[0])
        for i in range(n_clusters):
            client_params += pi[i] * cluster_centers[i]

        # Ajouter à la liste des paramètres aplatis
        all_flattened_params.append(client_params)

    # Combiner tous les paramètres avec les poids des clients
    aggregated_flattened = np.zeros_like(all_flattened_params[0])
    for client_id, flattened in enumerate(all_flattened_params):
        aggregated_flattened += weight_by_client[client_id] * flattened

    # Reconstruire les paramètres à leur forme d'origine
    aggregated_params = unflatten_parameters(aggregated_flattened, param_shapes, param_sizes)

    return aggregated_params

# Correction de la fonction load_real_data pour générer une erreur si les données ne peuvent pas être chargées
def load_real_data(n_clients, n_seeds_per_client, n_x, n_y, sequence_length):
    """
    Charge les données réelles pour chaque client (pays).

    Arguments:
    n_clients -- nombre de clients (pays)
    n_seeds_per_client -- nombre de seeds différentes pour chaque client
    n_x -- dimension d'entrée (nombre de features)
    n_y -- dimension de sortie (Consumption)
    sequence_length -- longueur de la séquence temporelle

    Returns:
    clients_data -- liste de tuples (X_train, Y_train) pour chaque client et seed

    Raises:
    FileNotFoundError: Si les fichiers de données ne sont pas trouvés
    ValueError: Si les données ne peuvent pas être traitées correctement
    """
    import os
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import StandardScaler

    clients_data = []
    countries = ["pays1", "pays2", "pays3"]  # Remplacez par les noms réels de vos pays

    for client_id in range(n_clients):
        if client_id >= len(countries):
            raise ValueError(f"Pas assez de pays définis pour {n_clients} clients")

        client_seeds_data = []
        country = countries[client_id]

        # Charger les données météo pour ce pays (12 mois)
        weather_data = []
        missing_weather_files = []

        for month in range(1, 13):
            file_path = f"/content/weather_{country}_month{month}.csv"  # Ajustez selon votre structure
            if not os.path.exists(file_path):
                missing_weather_files.append(file_path)
                continue

            try:
                df_month = pd.read_csv(file_path)
                weather_data.append(df_month)
            except Exception as e:
                raise FileNotFoundError(f"Erreur lors du chargement de {file_path}: {e}")

        if not weather_data:
            raise FileNotFoundError(f"Aucun fichier météo trouvé pour le pays {country}. Fichiers manquants: {missing_weather_files}")

        # Concaténer les données météo
        weather_df = pd.concat(weather_data, ignore_index=True)

        # Charger les données de consommation
        consumption_path = f"/content/consumption_{country}.csv"  # Ajustez selon votre structure
        if not os.path.exists(consumption_path):
            raise FileNotFoundError(f"Fichier de consommation non trouvé: {consumption_path}")

        try:
            consumption_df = pd.read_csv(consumption_path)
        except Exception as e:
            raise FileNotFoundError(f"Erreur lors du chargement de {consumption_path}: {e}")

        # Vérifier si les colonnes requises existent
        if "datetime" not in weather_df.columns:
            raise ValueError(f"Colonne 'datetime' manquante dans les données météo du pays {country}")

        if "MTU" not in consumption_df.columns:
            raise ValueError(f"Colonne 'MTU' manquante dans les données de consommation du pays {country}")

        # Convertir les colonnes de date/heure
        weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])
        consumption_df['MTU'] = pd.to_datetime(consumption_df['MTU'])

        # Fusionner sur les timestamps
        try:
            merged_df = pd.merge_asof(
                weather_df.sort_values('datetime'),
                consumption_df.sort_values('MTU'),
                left_on='datetime',
                right_on='MTU',
                direction='nearest'
            )
        except Exception as e:
            raise ValueError(f"Erreur lors de la fusion des données pour le pays {country}: {e}")

        if merged_df.empty:
            raise ValueError(f"La fusion a produit un DataFrame vide pour le pays {country}")

        # Sélectionner les features pertinentes
        features = ['temp', 'humidity', 'windspeed', 'feelslike', 'dew',
                   'precip', 'cloudcover', 'visibility', 'uvindex']

        # S'assurer que nous n'avons que n_x features et qu'elles existent
        features = features[:n_x]
        missing_features = [f for f in features if f not in merged_df.columns]
        if missing_features:
            raise ValueError(f"Features manquantes dans les données pour le pays {country}: {missing_features}")

        if 'Consumption' not in merged_df.columns:
            raise ValueError(f"Colonne 'Consumption' manquante dans les données fusionnées pour le pays {country}")

        # Vérifier les valeurs manquantes
        na_count = merged_df[features + ['Consumption']].isna().sum().sum()
        if na_count > 0:
            print(f"AVERTISSEMENT: {na_count} valeurs manquantes trouvées dans les données du pays {country}")
            # Remplir les valeurs manquantes ou échouer si le pourcentage est trop élevé
            na_percentage = na_count / (merged_df.shape[0] * (len(features) + 1)) * 100
            if na_percentage > 10:  # Si plus de 10% de valeurs manquantes
                raise ValueError(f"Trop de valeurs manquantes ({na_percentage:.2f}%) dans les données du pays {country}")

            # Remplir les valeurs manquantes (par exemple avec la moyenne)
            merged_df = merged_df.fillna(merged_df.mean())

        X_data = merged_df[features].values
        Y_data = merged_df[['Consumption']].values

        # Vérifier les dimensions des données
        if X_data.shape[0] < sequence_length:
            raise ValueError(f"Pas assez de données pour le pays {country}: {X_data.shape[0]} échantillons < {sequence_length} requis")

        # Normaliser les données
        scaler_X = StandardScaler()
        scaler_Y = StandardScaler()
        X_normalized = scaler_X.fit_transform(X_data)
        Y_normalized = scaler_Y.fit_transform(Y_data)

        # Créer plusieurs seeds en découpant les données différemment
        for seed in range(n_seeds_per_client):
            # Créer des séquences de données
            X_sequences, Y_sequences = create_sequences(
                X_normalized, Y_normalized, sequence_length, offset=seed
            )

            # Vérifier que nous avons suffisamment de données
            if len(X_sequences) == 0:
                raise ValueError(f"Pas assez de données pour créer des séquences pour le pays {country}")

            # Reformater pour LSTM (n_x, batch_size, sequence_length)
            X_train = np.transpose(X_sequences, (2, 0, 1))  # (n_x, batch_size, sequence_length)
            Y_train = np.transpose(Y_sequences, (1, 0, 2))  # (n_y, batch_size, sequence_length)

            client_seeds_data.append((X_train, Y_train))

        clients_data.append(client_seeds_data)

    if len(clients_data) == 0:
        raise ValueError("Aucune donnée client n'a pu être chargée")

    return clients_data

def create_sequences(X, Y, sequence_length, offset=0):
    """
    Crée des séquences temporelles à partir des données.

    Arguments:
    X -- données d'entrée, array de forme (n_samples, n_features)
    Y -- données de sortie, array de forme (n_samples, n_outputs)
    sequence_length -- longueur de la séquence
    offset -- décalage pour créer différentes séquences (pour les seeds)

    Returns:
    X_sequences -- séquences d'entrée
    Y_sequences -- séquences de sortie
    """
    X_sequences = []
    Y_sequences = []

    start_idx = offset % (sequence_length // 2) if offset > 0 else 0

    for i in range(start_idx, len(X) - sequence_length + 1, sequence_length // 2):
        X_sequences.append(X[i:i+sequence_length])
        Y_sequences.append(Y[i:i+sequence_length])

    return np.array(X_sequences), np.array(Y_sequences)

def create_test_dataset(clients_data, batch_size, n_x, n_y, sequence_length):
    """
    Crée un jeu de données de test à partir d'une fraction des données de chaque client.

    Arguments:
    clients_data -- données de tous les clients
    batch_size -- taille du batch pour le test
    n_x -- dimension d'entrée
    n_y -- dimension de sortie
    sequence_length -- longueur de la séquence temporelle

    Returns:
    X_test -- données de test, array de forme (n_x, batch_size, sequence_length)
    Y_test -- étiquettes de test, array de forme (n_y, batch_size, sequence_length)
    """
    # Extraire un échantillon de chaque client pour le test
    test_samples_X = []
    test_samples_Y = []

    for client_data in clients_data:
        # Prendre la première seed de chaque client
        X, Y = client_data[0]

        # Prendre 20% des échantillons pour le test
        n_samples = X.shape[1]
        n_test = min(batch_size // len(clients_data), n_samples // 5)

        test_samples_X.append(X[:, :n_test, :])
        test_samples_Y.append(Y[:, :n_test, :])

    # Concaténer les échantillons de test
    X_test = np.concatenate(test_samples_X, axis=1)
    Y_test = np.concatenate(test_samples_Y, axis=1)

    # S'assurer que la taille du batch est correcte
    if X_test.shape[1] > batch_size:
        X_test = X_test[:, :batch_size, :]
        Y_test = Y_test[:, :batch_size, :]

    return X_test, Y_test

def federated_main_local_clustering(
    n_clients=3,  # 3 pays comme clients
    n_seeds_per_client=5,
    n_epochs=30,
    n_clusters=3,
    n_a=64,
    n_x=9,  # Nombre de features d'entrée (temp, humidity, windspeed, etc.)
    n_y=1,  # Prédiction de Consumption
    batch_size=32,
    sequence_length=24,  # Par exemple, 24 heures
    num_transfer_epochs=5,
    learning_rate=0.01,
    use_synthetic_data=False  # Utilisez vos données réelles
):
    print("=== Lancement du Federated Learning avec compression markovienne locale ===")

    # Pour mesurer le temps d'exécution total
    start_time = time.time()

    # Charger les données réelles
    if not use_synthetic_data:
        print("Chargement des données réelles...")
        clients_data = load_real_data(n_clients, n_seeds_per_client, n_x, n_y, sequence_length)

        # Données de test (par exemple, utiliser un sous-ensemble des données)
        X_test, Y_test = create_test_dataset(clients_data, batch_size, n_x, n_y, sequence_length)

    else:
        # Ici, vous pourriez charger des données réelles
        # Pour l'instant, utiliser les données synthétiques
        print("Pas de données réelles disponibles, utilisation de données synthétiques...")
        clients_data = []

        for client_id in range(n_clients):
            client_seeds_data = []

            for seed in range(n_seeds_per_client):
                X_train = np.random.randn(n_x, batch_size, sequence_length)
                Y_train = np.zeros((n_y, batch_size, sequence_length))
                for t in range(sequence_length):
                    for i in range(batch_size):
                        class_idx = np.random.randint(0, n_y)
                        Y_train[class_idx, i, t] = 1
                client_seeds_data.append((X_train, Y_train))

            clients_data.append(client_seeds_data)

        # Données de test
        X_test = np.random.randn(n_x, batch_size//2, sequence_length)
        Y_test = np.zeros((n_y, batch_size//2, sequence_length))
        for t in range(sequence_length):
            for i in range(batch_size//2):
                class_idx = np.random.randint(0, n_y)
                Y_test[class_idx, i, t] = 1

    # 1. Entraînement local sur chaque client avec plusieurs seeds
    print("\n=== Entraînement local des clients avec plusieurs seeds ===")
    local_training_start = time.time()
    parameters_history_by_client_seed = []

    for client_id, client_seeds_data in enumerate(clients_data):
        print(f"\nClient {client_id+1}/{n_clients}")
        client_seeds_history = []

        for seed_id, (X_c, Y_c) in enumerate(client_seeds_data):
            print(f"  Seed {seed_id+1}/{n_seeds_per_client}")
            base_seed = client_id * 100 + seed_id  # Pour assurer l'unicité des seeds
            params, history, _ = train_lstm(X_c, Y_c, n_a, n_x, n_y,
                                          num_epochs=n_epochs, seed=base_seed)
            client_seeds_history.append(history)

        parameters_history_by_client_seed.append(client_seeds_history)

    local_training_time = time.time() - local_training_start
    print(f"Temps d'entraînement local total: {local_training_time:.2f} secondes")

    # 2. Clusterisation locale par client
    print("\n=== Clusterisation locale par client ===")
    clustering_start = time.time()

    client_clusters, param_shapes, param_sizes = cluster_local_by_client(
        parameters_history_by_client_seed, n_clusters=n_clusters
    )

    clustering_time = time.time() - clustering_start
    print(f"Temps de clusterisation locale: {clustering_time:.2f} secondes")

    # 3. Agrégation des clusters des clients
    print("\n=== Agrégation des clusters des clients ===")
    aggregation_start = time.time()

    # Agréger les clusters pour obtenir le modèle global
    local_compressed_params = aggregate_client_clusters(
        client_clusters, param_shapes, param_sizes, n_clients
    )

    aggregation_time = time.time() - aggregation_start
    print(f"Temps d'agrégation: {aggregation_time:.2f} secondes")

    # 4. Évaluation des performances du modèle
    print("\n=== Évaluation des performances ===")

    # 4.1 Modèle global obtenu par agrégation des clusters locaux
    local_compressed_acc, local_compressed_loss = evaluate_lstm(X_test, Y_test, local_compressed_params)
    print(f"Modèle compressé local - Accuracy: {local_compressed_acc:.4f}, Loss: {local_compressed_loss:.4f}")

    # 4.2 Obtenir le modèle original (méthode FedAvg standard) pour comparaison
    # Extraire les derniers paramètres de chaque client pour chaque seed
    last_params_by_client = []
    for client_id in range(n_clients):
        client_seed_last_params = []
        for seed_id in range(n_seeds_per_client):
            client_seed_last_params.append(parameters_history_by_client_seed[client_id][seed_id][-1])

        # Moyenne des paramètres sur toutes les seeds pour ce client
        avg_params = {}
        for key in client_seed_last_params[0].keys():
            avg_params[key] = np.mean([params[key] for params in client_seed_last_params], axis=0)

        last_params_by_client.append(avg_params)

    # Moyenne des paramètres sur tous les clients (FedAvg standard)
    original_params = {}
    for key in last_params_by_client[0].keys():
        original_params[key] = np.mean([params[key] for params in last_params_by_client], axis=0)

    original_acc, original_loss = evaluate_lstm(X_test, Y_test, original_params)
    print(f"Original (FedAvg) - Accuracy: {original_acc:.4f}, Loss: {original_loss:.4f}")

    # 5. Calculer la taille de transmission des paramètres
    original_size = calculate_transmission_size(original_params)

    # Pour la méthode locale, nous transmettons les centres de clusters et matrices de transition
    # de chaque client au serveur
    local_compressed_size = calculate_transmission_size(parameters=None, client_centers=client_clusters)

    print(f"Taille originale (FedAvg): {original_size/1024:.2f} KB")
    print(f"Taille compressée (locale): {local_compressed_size/1024:.2f} KB")
    print(f"Réduction: {(1 - local_compressed_size/original_size)*100:.2f}%")

    # 6. Réentraînement à partir du modèle compressé (transfer learning)
    print("\n=== Réentraînement à partir du modèle compressé local ===")
    transfer_start = time.time()

    local_transfer_params, local_transfer_loss_history, local_transfer_acc_history = transfer_learning(
        X_train=X_test, Y_train=Y_test,
        X_test=X_test, Y_test=Y_test,
        source_parameters=local_compressed_params,
        n_a=n_a, n_x=n_x, n_y=n_y,
        num_epochs=num_transfer_epochs,
        learning_rate=learning_rate
    )

    transfer_time = time.time() - transfer_start
    print(f"Temps de transfer learning: {transfer_time:.2f} secondes")

    local_transfer_acc = local_transfer_acc_history[-1]
    local_transfer_loss = local_transfer_loss_history[-1]

    # 7. Simulation de la méthode traditionnelle (transmission complète des paramètres)
    print("\n=== Simulation de la méthode traditionnelle (FedAvg) ===")
    full_params_avg = original_params  # Déjà calculé précédemment

    # Évaluer les performances de FedAvg après transfer learning
    full_transfer_params, full_transfer_loss_history, full_transfer_acc_history = transfer_learning(
        X_train=X_test, Y_train=Y_test,
        X_test=X_test, Y_test=Y_test,
        source_parameters=full_params_avg,
        n_a=n_a, n_x=n_x, n_y=n_y,
        num_epochs=num_transfer_epochs,
        learning_rate=learning_rate
    )

    full_transfer_acc = full_transfer_acc_history[-1]
    full_transfer_loss = full_transfer_loss_history[-1]

    print(f"FedAvg après transfer - Accuracy: {full_transfer_acc:.4f}, Loss: {full_transfer_loss:.4f}")

    # 8. Analyse de la complexité computationnelle
    complexity_dict = analyze_computational_complexity_local(
        n_clients, n_seeds_per_client, n_a, n_x, n_y, n_epochs, n_clusters, sequence_length, batch_size
    )

    # 9. Visualisations
    # Visualiser les résultats de performance
    visualize_results_local(
        original_acc, local_compressed_acc, local_transfer_acc, full_transfer_acc,
        original_loss, local_compressed_loss, local_transfer_loss, full_transfer_loss
    )

    # Visualiser les matrices de transition de chaque client
    visualize_transition_matrices_local(client_clusters)

    # Visualiser l'économie de bande passante
    visualize_bandwidth_savings(original_size, local_compressed_size)

    # Visualiser la complexité computationnelle
    visualize_computational_complexity_local(complexity_dict)

    # Calculer le temps total d'exécution
    total_time = time.time() - start_time
    print(f"\nTemps total d'exécution: {total_time:.2f} secondes")

    # Résumé des résultats
    print("\n=== Résumé ===")
    print(f"Taux de compression : {original_size/local_compressed_size:.2f}x")
    print(f"Économie de bande passante : {(1 - local_compressed_size/original_size)*100:.2f}%")
    print(f"Perte relative de performance (compression) : {(original_acc - local_compressed_acc)/original_acc*100:.2f}%")
    print(f"Récupération après transfert : {(local_transfer_acc - local_compressed_acc)/local_compressed_acc*100:.2f}%")
    print(f"Efficacité computationnelle : {complexity_dict['Efficiency Ratio']:.2f}x")
    print(f"Comparaison avec FedAvg après transfert: {(local_transfer_acc/full_transfer_acc)*100:.2f}% des performances")

    # Collecter tous les résultats dans un dictionnaire
    results = {
        "original_acc": original_acc,
        "local_compressed_acc": local_compressed_acc,
        "local_transfer_acc": local_transfer_acc,
        "full_transfer_acc": full_transfer_acc,
        "original_loss": original_loss,
        "local_compressed_loss": local_compressed_loss,
        "local_transfer_loss": local_transfer_loss,
        "full_transfer_loss": full_transfer_loss,
        "original_size": original_size,
        "local_compressed_size": local_compressed_size,
        "bandwidth_saving": (1 - local_compressed_size/original_size)*100,
        "compression_ratio": original_size/local_compressed_size,
        "performance_loss": (original_acc - local_compressed_acc)/original_acc*100,
        "performance_recovery": (local_transfer_acc - local_compressed_acc)/local_compressed_acc*100,
        "performance_vs_full": (local_transfer_acc/full_transfer_acc)*100,
        "computational_efficiency": complexity_dict['Efficiency Ratio'],
        "total_time": total_time,
        "local_training_time": local_training_time,
        "clustering_time": clustering_time,
        "aggregation_time": aggregation_time,
        "transfer_time": transfer_time,
        "complexity": complexity_dict,
        "n_clients": n_clients,
        "n_seeds_per_client": n_seeds_per_client,
        "n_epochs": n_epochs,
        "n_clusters": n_clusters
    }

    return results

def main():
    """
    Fonction principale pour comparer les approches de clusterisation globale et locale.
    """
    print("=" * 80)
    print("COMPARAISON DES MÉTHODES DE COMPRESSION MARKOVIENNE DES PARAMÈTRES LSTM")
    print("=" * 80)
    setup_google_colab()
    # Paramètres communs
    params = {
        "n_clients": 3,           # Nombre de clients
        "n_epochs": 50,            # Nombre d'époques pour l'entraînement
        "n_clusters": 3,          # Nombre de clusters pour la compression
        "n_a": 32,                # Dimension cachée
        "n_x": 8,                 # Dimension d'entrée
        "n_y": 5,                 # Nombre de classes
        "batch_size": 32,         # Taille du batch
        "sequence_length": 10,    # Longueur de séquence
        "num_transfer_epochs": 3, # Époques pour le transfer learning
        "learning_rate": 0.01,    # Taux d'apprentissage
        "use_synthetic_data": True # Utiliser des données synthétiques
    }

    # 1. Exécuter l'approche de clusterisation globale
    print("\n\n" + "=" * 80)
    print("APPROCHE GLOBALE : Clusterisation sur l'ensemble des clients")
    print("=" * 80)

    results_global = federated_main(**params)

    # 2. Exécuter l'approche de clusterisation locale
    print("\n\n" + "=" * 80)
    print("APPROCHE LOCALE : Clusterisation individuelle par client")
    print("=" * 80)

    # Ajouter le paramètre spécifique à l'approche locale
    params_local = params.copy()
    params_local["n_seeds_per_client"] = 5  # Nombre de seeds par client

    results_local = federated_main_local_clustering(**params_local)

    # 3. Comparer les résultats
    compare_approaches(results_global, results_local)

    return results_global, results_local
def analyze_computational_complexity_local(n_clients, n_seeds_per_client, n_a, n_x, n_y, n_epochs,
                                          n_clusters, sequence_length, batch_size):
    """
    Analyse la complexité computationnelle du processus d'apprentissage fédéré avec clusterisation locale.
    """
    complexity_dict = {}

    # Dimension des paramètres LSTM
    d = n_a**2 + n_a*n_x + n_a*n_y + n_a + n_y*n_a + n_y  # Dimension totale des paramètres

    # 1. Complexité de l'entraînement local sur un client pour chaque seed (par époque)
    forward_complexity = batch_size * sequence_length * (n_a**2 + n_a*n_x + n_a*n_y)
    backward_complexity = batch_size * sequence_length * (n_a**2 + n_a*n_x + n_a*n_y)
    client_seed_training_complexity = n_epochs * (forward_complexity + backward_complexity)
    client_training_complexity = n_seeds_per_client * client_seed_training_complexity

    complexity_dict["Local Training (per client)"] = client_training_complexity
    complexity_dict["Local Training (per seed)"] = client_seed_training_complexity
    complexity_dict["Local Training (all clients)"] = n_clients * client_training_complexity

    # 2. Complexité de la clusterisation locale
    kmeans_iterations = 100  # Hypothèse pour le nombre d'itérations K-means
    n_samples = n_seeds_per_client * n_epochs  # Nombre d'échantillons à clusteriser

    # Complexité de K-means pour un client
    clustering_complexity_per_client = kmeans_iterations * n_samples * n_clusters * d
    clustering_complexity_all_clients = n_clients * clustering_complexity_per_client

    complexity_dict["Local Clustering (per client)"] = clustering_complexity_per_client
    complexity_dict["Local Clustering (all clients)"] = clustering_complexity_all_clients

    # 3. Complexité du calcul des matrices de transition locales
    transition_matrix_complexity_per_client = n_seeds_per_client * n_epochs * n_clusters**2
    transition_matrix_complexity_all_clients = n_clients * transition_matrix_complexity_per_client

    complexity_dict["Transition Matrix Computation (per client)"] = transition_matrix_complexity_per_client
    complexity_dict["Transition Matrix Computation (all clients)"] = transition_matrix_complexity_all_clients

    # 4. Complexité de la communication client-serveur
    cluster_centers_size = n_clusters * d  # Taille des centres de clusters
    transition_matrix_size = n_clusters * n_clusters  # Taille de la matrice de transition

    communication_complexity_per_client = cluster_centers_size + transition_matrix_size
    communication_complexity_all_clients = n_clients * communication_complexity_per_client

    traditional_communication_per_client = d
    traditional_communication_all_clients = n_clients * traditional_communication_per_client

    complexity_dict["Communication (proposed, per client)"] = communication_complexity_per_client
    complexity_dict["Communication (proposed, all clients)"] = communication_complexity_all_clients
    complexity_dict["Communication (traditional, per client)"] = traditional_communication_per_client
    complexity_dict["Communication (traditional, all clients)"] = traditional_communication_all_clients
    complexity_dict["Communication Reduction Ratio"] = traditional_communication_all_clients / communication_complexity_all_clients

    # 5. Complexité de l'agrégation au serveur
    distribution_stationary_complexity = n_clients * n_clusters * n_clusters * 100
    aggregation_complexity_proposed = distribution_stationary_complexity + n_clients * d

    aggregation_complexity_traditional = n_clients * d

    complexity_dict["Server Aggregation (proposed)"] = aggregation_complexity_proposed
    complexity_dict["Server Aggregation (traditional)"] = aggregation_complexity_traditional

    # 6. Complexité du transfer learning
    transfer_epochs = 5  # Hypothèse
    transfer_complexity = transfer_epochs * (forward_complexity + backward_complexity)

    complexity_dict["Transfer Learning"] = transfer_complexity

    # 7. Complexité totale des approches
    proposed_approach_complexity = (
        n_clients * client_training_complexity +  # Entraînement local
        clustering_complexity_all_clients +       # Clusterisation locale
        transition_matrix_complexity_all_clients + # Calcul des matrices de transition
        communication_complexity_all_clients +    # Communication client-serveur
        aggregation_complexity_proposed +         # Agrégation au serveur
        transfer_complexity                       # Transfer learning
    )

    traditional_approach_complexity = (
        n_clients * client_training_complexity +  # Entraînement local (même coût)
        traditional_communication_all_clients +   # Communication client-serveur
        aggregation_complexity_traditional +      # Agrégation au serveur
        transfer_complexity                       # Transfer learning (même coût)
    )

    complexity_dict["Total (Proposed Approach)"] = proposed_approach_complexity
    complexity_dict["Total (Traditional Approach)"] = traditional_approach_complexity

    efficiency_ratio = traditional_approach_complexity / proposed_approach_complexity
    complexity_dict["Efficiency Ratio"] = efficiency_ratio

    # 8. Analyse par composante (pourcentage du temps total)
    total_proposed = proposed_approach_complexity

    complexity_dict["Local Training (% of total)"] = n_clients * client_training_complexity / total_proposed * 100
    complexity_dict["Local Clustering (% of total)"] = clustering_complexity_all_clients / total_proposed * 100
    complexity_dict["Transition Matrix (% of total)"] = transition_matrix_complexity_all_clients / total_proposed * 100
    complexity_dict["Communication (% of total)"] = communication_complexity_all_clients / total_proposed * 100
    complexity_dict["Server Aggregation (% of total)"] = aggregation_complexity_proposed / total_proposed * 100
    complexity_dict["Transfer Learning (% of total)"] = transfer_complexity / total_proposed * 100

    return complexity_dict

def transfer_learning(X_train, Y_train, X_test, Y_test, source_parameters, n_a, n_x, n_y, num_epochs=5, learning_rate=0.001):
    """
    Effectue un transfert d'apprentissage à partir des paramètres source.
    """
    # Utiliser les paramètres source comme initialisation
    parameters = copy.deepcopy(source_parameters)

    # Initialiser Adam
    v, s = initialize_adam_for_lstm(parameters)
    t = 0  # Compteur pour Adam

    loss_history = []
    accuracy_history = []

    for epoch in range(num_epochs):
        print(f"Époque de transfert {epoch+1}/{num_epochs}")

        # Forward pass
        a0 = np.zeros((n_a, X_train.shape[1]))
        a, y_pred, c, caches = lstm_forward(X_train, a0, parameters)

        # Calcul de la perte (cross-entropy)
        loss = -np.sum(Y_train * np.log(y_pred + 1e-8)) / (Y_train.shape[1] * Y_train.shape[2])

        # Initialisation du gradient de sortie
        da = np.zeros_like(a)

        # Calculer les gradients pour Wy et by
        dWy = np.zeros_like(parameters["Wy"])
        dby = np.zeros_like(parameters["by"])

        # Pour chaque pas de temps, calculer le gradient
        for t_idx in range(Y_train.shape[2]):
            # Gradient de la cross-entropy
            dy = y_pred[:, :, t_idx] - Y_train[:, :, t_idx]
            # Accumuler les gradients pour Wy et by
            dWy += np.dot(dy, a[:, :, t_idx].T)
            dby += np.sum(dy, axis=1, keepdims=True)
            # Gradient par rapport à a
            da[:, :, t_idx] = np.dot(parameters["Wy"].T, dy)

        # Backward pass pour les autres paramètres
        lstm_gradients = lstm_backward(da, caches)

        # Combiner tous les gradients
        gradients = lstm_gradients.copy()
        gradients["dWy"] = dWy
        gradients["dby"] = dby

        # Mise à jour des paramètres avec Adam
        t += 1
        parameters, v, s = update_parameters_with_adam_for_lstm(parameters, gradients, v, s, t, learning_rate)

        # Évaluer sur l'ensemble de test
        accuracy, test_loss = evaluate_lstm(X_test, Y_test, parameters)
        loss_history.append(test_loss)
        accuracy_history.append(accuracy)

        print(f"Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}")

    return parameters, loss_history, accuracy_history

def visualize_results_local(original_acc, local_compressed_acc, local_transfer_acc, full_transfer_acc,
                           original_loss, local_compressed_loss, local_transfer_loss, full_transfer_loss):
    """
    Visualise les résultats de l'expérience avec clusterisation locale.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Graphique des précisions
    labels = ['Original', 'Compressé Local', 'Transfert Local', 'FedAvg+Transfert']
    accuracies = [original_acc, local_compressed_acc, local_transfer_acc, full_transfer_acc]
    colors = ['blue', 'orange', 'green', 'red']
    ax1.bar(labels, accuracies, color=colors)
    ax1.set_ylabel('Précision')
    ax1.set_title('Comparaison des précisions')

    # Ajouter les valeurs numériques sur les barres
    for i, v in enumerate(accuracies):
        ax1.text(i, v + 0.01, f"{v:.3f}", ha='center')

    # Graphique des pertes
    losses = [original_loss, local_compressed_loss, local_transfer_loss, full_transfer_loss]
    ax2.bar(labels, losses, color=colors)
    ax2.set_ylabel('Perte')
    ax2.set_title('Comparaison des pertes')

    # Ajouter les valeurs numériques sur les barres
    for i, v in enumerate(losses):
        ax2.text(i, v + 0.1, f"{v:.3f}", ha='center')

    plt.tight_layout()
    plt.savefig('resultats_compression_locale_lstm.png')
    plt.show()

def visualize_transition_matrices_local(client_clusters):
    """
    Visualise les matrices de transition de chaque client.
    """
    n_clients = len(client_clusters)

    # Déterminer la disposition optimale des sous-figures
    n_cols = min(3, n_clients)
    n_rows = (n_clients + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))

    # Gérer le cas d'un seul client (axes n'est pas un tableau)
    if n_clients == 1:
        axes = np.array([axes])

    # Aplatir le tableau d'axes pour itération facile
    if n_rows * n_cols > 1:
        axes = axes.flatten()

    for i, (_, transition_matrix) in enumerate(client_clusters):
        if i < len(axes):
            ax = axes[i]
            im = ax.imshow(transition_matrix, cmap='viridis', interpolation='none')
            ax.set_title(f'Client {i+1}')
            ax.set_xlabel('Cluster de destination')
            ax.set_ylabel('Cluster de départ')

            # Ajouter les valeurs sur la figure
            for row in range(transition_matrix.shape[0]):
                for col in range(transition_matrix.shape[1]):
                    ax.text(col, row, f'{transition_matrix[row, col]:.2f}',
                             ha='center', va='center',
                             color='white' if transition_matrix[row, col] > 0.5 else 'black')

    # Masquer les axes supplémentaires s'il y en a
    for i in range(n_clients, len(axes)):
        axes[i].axis('off')

    # Ajouter une barre de couleur commune
    fig.colorbar(im, ax=axes.tolist(), label='Probabilité de transition')

    plt.tight_layout()
    plt.savefig('matrices_transition_locales.png')
    plt.show()

def visualize_computational_complexity_local(complexity_dict):
    """
    Visualise la complexité computationnelle pour l'approche de clusterisation locale.
    """
    # Extraction des composantes principales pour la visualisation
    components = ["Local Training (per client)", "Local Clustering (per client)",
                  "Transition Matrix Computation (per client)",
                  "Server Aggregation (proposed)", "Transfer Learning"]

    # Vérifier que toutes les clés existent
    for comp in components:
        if comp not in complexity_dict:
            print(f"Attention: Clé '{comp}' non trouvée dans complexity_dict")
            # Remplacer par une valeur par défaut pour éviter l'erreur
            complexity_dict[comp] = 0

    values = [complexity_dict[comp] for comp in components]

    # Normalisation pour une meilleure visualisation
    total = sum(values)
    if total > 0:
        normalized_values = np.array(values) / total * 100
    else:
        normalized_values = np.zeros_like(values)

    # Création du graphique
    plt.figure(figsize=(12, 6))
    bars = plt.bar(components, normalized_values, color=['blue', 'green', 'orange', 'red', 'purple'])

    # Ajout des valeurs en pourcentage
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                 f'{height:.1f}%', ha='center', va='bottom')

    plt.title('Décomposition de la complexité computationnelle (Clusterisation Locale)')
    plt.ylabel('Pourcentage du temps de calcul total')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('decomposition_complexite_locale.png')
    plt.show()

    # Comparaison des approches
    plt.figure(figsize=(10, 6))
    ratio = complexity_dict.get("Efficiency Ratio", 1.0)  # Valeur par défaut de 1.0
    plt.bar(['Approche traditionnelle (FedAvg)', 'Approche proposée (Locale)'],
            [100, 100/ratio if ratio > 0 else 0],
            color=['gray', 'green'])
    plt.title(f'Comparaison de l\'efficacité (Traditionnel / Proposé = {ratio:.2f}x)')
    plt.ylabel('Complexité relative (%)')
    plt.ylim(0, max(100, 100/ratio if ratio > 0 else 0) * 1.1)
    plt.tight_layout()
    plt.savefig('comparaison_complexite_locale.png')
    plt.show()

def visualize_bandwidth_savings(original_size, compressed_size):
    """
    Visualise l'économie de bande passante.
    """
    # Vérifier que original_size n'est pas zéro pour éviter division par zéro
    if original_size <= 0:
        print("AVERTISSEMENT: La taille originale est nulle ou négative, impossible de calculer le pourcentage d'économie.")
        saved_percentage = 0
    else:
        saved_percentage = (1 - compressed_size / original_size) * 100

    plt.figure(figsize=(10, 6))

    # Barres de taille
    bars = plt.bar(['Paramètres originaux', 'Paramètres compressés'],
                   [original_size/1024 if original_size > 0 else 0, compressed_size/1024],
                   color=['blue', 'green'])

    # Ajouter valeurs numériques
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                 f'{height:.2f} KB', ha='center', va='bottom')

    if original_size > 0:
        plt.title(f'Économie de bande passante: {saved_percentage:.2f}%')
    else:
        plt.title('Économie de bande passante: Non calculable (taille originale nulle)')
    plt.ylabel('Taille (KB)')
    plt.tight_layout()
    plt.savefig('economies_bande_passante.png')
    plt.show()

    # Diagramme circulaire pour visualiser les proportions - uniquement si les valeurs sont valides
    if original_size > 0 and compressed_size < original_size:  # Cas normal: compression réduit la taille
        plt.figure(figsize=(8, 8))
        plt.pie([compressed_size, original_size - compressed_size],
                labels=['Utilisé', 'Économisé'],
                colors=['green', 'lightgray'],
                autopct='%1.1f%%',
                startangle=90,
                explode=(0, 0.1))
        plt.axis('equal')
        plt.title(f'Économie de bande passante: {saved_percentage:.2f}%')
        plt.tight_layout()
        plt.savefig('pourcentage_economie.png')
        plt.show()
    elif original_size > 0 and compressed_size > original_size:  # Cas où la compression augmente la taille
        plt.figure(figsize=(8, 8))
        plt.pie([original_size, compressed_size - original_size],
                labels=['Taille originale', 'Surcoût de compression'],
                colors=['blue', 'red'],
                autopct='%1.1f%%',
                startangle=90,
                explode=(0, 0.1))
        plt.axis('equal')
        plt.title(f'Augmentation de la taille: {-saved_percentage:.2f}%')
        plt.tight_layout()
        plt.savefig('pourcentage_surcout.png')
        plt.show()
    # Pas de diagramme si original_size est nul ou négatif

def update_with_transition_matrices(client_transition_matrices, global_kmeans_model, initial_global_params, param_shapes, param_sizes):
    """
    Met à jour le modèle global en utilisant uniquement les matrices de transition des clients.

    Arguments:
    client_transition_matrices -- liste des matrices de transition de chaque client
    global_kmeans_model -- modèle KMeans global utilisé pour la clusterisation
    initial_global_params -- paramètres initiaux obtenus par FedAvg
    param_shapes -- formes des paramètres
    param_sizes -- tailles des paramètres aplatis

    Returns:
    updated_global_params -- paramètres globaux mis à jour
    """
    n_clients = len(client_transition_matrices)
    n_clusters = global_kmeans_model.cluster_centers_.shape[0]

    # Agréger les matrices de transition des clients
    global_transition_matrix = np.zeros((n_clusters, n_clusters))
    for client_id in range(n_clients):
        global_transition_matrix += client_transition_matrices[client_id]

    # Normaliser la matrice de transition globale
    for i in range(n_clusters):
        row_sum = np.sum(global_transition_matrix[i])
        if row_sum > 0:
            global_transition_matrix[i] = global_transition_matrix[i] / row_sum
        else:
            global_transition_matrix[i] = np.ones(n_clusters) / n_clusters

    # Calculer la distribution stationnaire de la matrice de transition
    pi = np.ones(n_clusters) / n_clusters  # Distribution initiale uniforme
    for _ in range(100):  # Nombre d'itérations arbitraire pour convergence
        pi_new = np.dot(pi, global_transition_matrix)
        if np.allclose(pi, pi_new):
            break
        pi = pi_new

    # Créer le nouveau modèle global en pondérant les centres par la distribution stationnaire
    weighted_centers = np.zeros_like(global_kmeans_model.cluster_centers_[0])
    for i in range(n_clusters):
        weighted_centers += pi[i] * global_kmeans_model.cluster_centers_[i]

    # Convertir les paramètres aplatis en structure de dictionnaire
    updated_global_params = unflatten_parameters(weighted_centers, param_shapes, param_sizes)

    return updated_global_params
def visualize_communication_rounds(communication_results):
    """
    Visualise les résultats pour chaque cycle de communication.

    Arguments:
    communication_results -- liste de dictionnaires contenant les résultats pour chaque cycle
    """
    n_rounds = len(communication_results)
    cycles = [result["cycle"] + 1 for result in communication_results]  # +1 pour commencer à 1

    # Extraction des données
    accuracies = [result["accuracy"] for result in communication_results]
    losses = [result["loss"] for result in communication_results]
    traditional_bw = [result["traditional_bandwidth"]/1024 for result in communication_results]  # KB
    proposed_bw = [result["proposed_bandwidth"]/1024 for result in communication_results]  # KB

    # Créer une figure avec 2 sous-graphiques
    fig, axes = plt.subplots(2, 1, figsize=(12, 10))

    # Graphique des performances
    ax1 = axes[0]
    color = 'tab:blue'
    ax1.set_xlabel('Cycle de communication')
    ax1.set_ylabel('Précision', color=color)
    ax1.plot(cycles, accuracies, 'o-', color=color, label='Précision')
    ax1.tick_params(axis='y', labelcolor=color)

    # Ajouter la perte sur le même graphique avec un axe y secondaire
    ax1_bis = ax1.twinx()
    color = 'tab:red'
    ax1_bis.set_ylabel('Perte', color=color)
    ax1_bis.plot(cycles, losses, 's-', color=color, label='Perte')
    ax1_bis.tick_params(axis='y', labelcolor=color)

    # Ajouter un titre et une légende
    ax1.set_title('Évolution des performances par cycle de communication')
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax1_bis.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

    # Graphique de la bande passante
    ax2 = axes[1]
    ax2.set_xlabel('Cycle de communication')
    ax2.set_ylabel('Bande passante (KB)')
    ax2.bar(np.array(cycles) - 0.2, traditional_bw, width=0.4, color='gray', label='Traditionnelle (FedAvg)')
    ax2.bar(np.array(cycles) + 0.2, proposed_bw, width=0.4, color='green', label='Proposée (Matrices de transition)')

    # Ajouter les chiffres sur les barres
    for i, (trad, prop) in enumerate(zip(traditional_bw, proposed_bw)):
        ax2.text(cycles[i] - 0.2, trad + 0.5, f'{trad:.1f}', ha='center')
        ax2.text(cycles[i] + 0.2, prop + 0.5, f'{prop:.1f}', ha='center')

    # Ajouter pourcentage d'économie pour les cycles > 0
    for i in range(1, n_rounds):
        saving = (1 - proposed_bw[i]/traditional_bw[i]) * 100
        ax2.text(cycles[i], max(traditional_bw[i], proposed_bw[i]) * 1.1,
                 f'Économie: {saving:.1f}%', ha='center', fontweight='bold')

    ax2.set_title('Comparaison de la bande passante par cycle de communication')
    ax2.legend()

    # Ajuster la mise en page
    plt.tight_layout()
    plt.savefig('resultats_federated_two_phase.png')
    plt.show()

def visualize_communication_rounds_comparison(communication_results):
    """
    Visualise la comparaison des deux méthodes pour chaque cycle de communication.

    Arguments:
    communication_results -- liste de dictionnaires contenant les résultats pour chaque cycle
    """
    n_rounds = len(communication_results)
    cycles = [result["cycle"] + 1 for result in communication_results]  # +1 pour commencer à 1

    # Extraction des données
    proposed_acc = [result["proposed_accuracy"] for result in communication_results]
    proposed_loss = [result["proposed_loss"] for result in communication_results]
    fedavg_acc = [result["fedavg_accuracy"] for result in communication_results]
    fedavg_loss = [result["fedavg_loss"] for result in communication_results]
    traditional_bw = [result["traditional_bandwidth"]/1024 for result in communication_results]  # KB
    proposed_bw = [result["proposed_bandwidth"]/1024 for result in communication_results]  # KB

    # Créer une figure avec 3 sous-graphiques
    fig, axes = plt.subplots(3, 1, figsize=(12, 15))

    # Graphique de précision
    ax1 = axes[0]
    ax1.set_xlabel('Cycle de communication')
    ax1.set_ylabel('Précision')
    ax1.plot(cycles, proposed_acc, 'o-', color='blue', label='Méthode proposée')
    ax1.plot(cycles, fedavg_acc, 's-', color='red', label='FedAvg traditionnel')

    # Ajouter les valeurs sur les points
    for i, (acc_p, acc_f) in enumerate(zip(proposed_acc, fedavg_acc)):
        ax1.text(cycles[i], acc_p + 0.01, f'{acc_p:.3f}', ha='center')
        ax1.text(cycles[i], acc_f - 0.02, f'{acc_f:.3f}', ha='center')

    ax1.set_title('Comparaison de la précision')
    ax1.legend()
    ax1.grid(alpha=0.3)

    # Graphique de perte
    ax2 = axes[1]
    ax2.set_xlabel('Cycle de communication')
    ax2.set_ylabel('Perte')
    ax2.plot(cycles, proposed_loss, 'o-', color='blue', label='Méthode proposée')
    ax2.plot(cycles, fedavg_loss, 's-', color='red', label='FedAvg traditionnel')

    # Ajouter les valeurs sur les points
    for i, (loss_p, loss_f) in enumerate(zip(proposed_loss, fedavg_loss)):
        ax2.text(cycles[i], loss_p + 0.02, f'{loss_p:.3f}', ha='center')
        ax2.text(cycles[i], loss_f - 0.04, f'{loss_f:.3f}', ha='center')

    ax2.set_title('Comparaison de la perte')
    ax2.legend()
    ax2.grid(alpha=0.3)

    # Graphique de la bande passante
    ax3 = axes[2]
    ax3.set_xlabel('Cycle de communication')
    ax3.set_ylabel('Bande passante (KB)')

    # Largeur des barres
    width = 0.35
    x = np.array(cycles)

    # Barres
    ax3.bar(x - width/2, traditional_bw, width, color='red', label='FedAvg traditionnel')
    ax3.bar(x + width/2, proposed_bw, width, color='blue', label='Méthode proposée')

    # Ajouter les chiffres sur les barres
    for i, (trad, prop) in enumerate(zip(traditional_bw, proposed_bw)):
        ax3.text(cycles[i] - width/2, trad + max(traditional_bw)/40, f'{trad:.1f}', ha='center')
        ax3.text(cycles[i] + width/2, prop + max(traditional_bw)/40, f'{prop:.1f}', ha='center')

    # Ajouter pourcentage d'économie pour les cycles > 0
    for i in range(1, n_rounds):
        saving = (1 - proposed_bw[i]/traditional_bw[i]) * 100
        ax3.text(cycles[i], max(traditional_bw[i], proposed_bw[i]) * 1.1,
                 f'Économie: {saving:.1f}%', ha='center', fontweight='bold')

    ax3.set_title('Comparaison de la bande passante')
    ax3.legend()

    # Ajuster la mise en page
    plt.tight_layout()
    plt.savefig('resultats_comparaison_federated.png')
    plt.show()

    # Créer un second graphique pour une comparaison directe des performances
    plt.figure(figsize=(12, 6))

    # Pour la précision
    plt.subplot(1, 2, 1)
    x = np.arange(n_rounds)
    width = 0.35
    plt.bar(x - width/2, proposed_acc, width, label='Méthode proposée', color='blue')
    plt.bar(x + width/2, fedavg_acc, width, label='FedAvg traditionnel', color='red')

    # Ajouter les valeurs sur les barres
    for i, (acc_p, acc_f) in enumerate(zip(proposed_acc, fedavg_acc)):
        plt.text(i - width/2, acc_p + 0.01, f'{acc_p:.3f}', ha='center')
        plt.text(i + width/2, acc_f + 0.01, f'{acc_f:.3f}', ha='center')

    plt.xlabel('Cycle de communication')
    plt.ylabel('Précision')
    plt.title('Comparaison de la précision')
    plt.xticks(x, [f'Cycle {i+1}' for i in range(n_rounds)])
    plt.legend()

    # Pour la perte
    plt.subplot(1, 2, 2)
    plt.bar(x - width/2, proposed_loss, width, label='Méthode proposée', color='blue')
    plt.bar(x + width/2, fedavg_loss, width, label='FedAvg traditionnel', color='red')

    # Ajouter les valeurs sur les barres
    for i, (loss_p, loss_f) in enumerate(zip(proposed_loss, fedavg_loss)):
        plt.text(i - width/2, loss_p + 0.02, f'{loss_p:.3f}', ha='center')
        plt.text(i + width/2, loss_f + 0.02, f'{loss_f:.3f}', ha='center')

    plt.xlabel('Cycle de communication')
    plt.ylabel('Perte')
    plt.title('Comparaison de la perte')
    plt.xticks(x, [f'Cycle {i+1}' for i in range(n_rounds)])
    plt.legend()

    plt.tight_layout()
    plt.savefig('resultats_performances_comparaison.png')
    plt.show()

def federated_main_two_phase(
    n_clients=3, n_epochs=50, n_clusters=3,
    n_a=64, n_x=10, n_y=5,
    batch_size=32, sequence_length=10,
    num_transfer_epochs=5, learning_rate=0.01,
    n_communication_rounds=6,  # Nombre total de cycles de communication
    use_synthetic_data=True
):
    """
    Pipeline complet de Federated Learning avec approche en deux phases :
    Phase 1: Initialisation avec FedAvg
    Phase 2: Mises à jour avec matrices de transition
    Compare aussi avec la méthode traditionnelle FedAvg à chaque cycle
    """
    print("=== Lancement du Federated Learning en deux phases ===")
    start_time = time.time()

    # Génération/chargement des données
    if use_synthetic_data:
        print("Génération des données synthétiques...")
        clients_data = []
        for seed in range(n_clients):
            X_train = np.random.randn(n_x, batch_size, sequence_length)
            Y_train = np.zeros((n_y, batch_size, sequence_length))
            for t in range(sequence_length):
                for i in range(batch_size):
                    class_idx = np.random.randint(0, n_y)
                    Y_train[class_idx, i, t] = 1
            clients_data.append((X_train, Y_train))

        # Données de test
        X_test = np.random.randn(n_x, batch_size//2, sequence_length)
        Y_test = np.zeros((n_y, batch_size//2, sequence_length))
        for t in range(sequence_length):
            for i in range(batch_size//2):
                class_idx = np.random.randint(0, n_y)
                Y_test[class_idx, i, t] = 1
    else:
        # Code pour les données réelles
        print("Chargement des données réelles...")
        clients_data = load_real_data(n_clients, 1, n_x, n_y, sequence_length)
        # Extraire seulement la première seed pour chaque client
        clients_data = [client_seeds[0] for client_seeds in clients_data]
        # Créer des données de test
        X_test, Y_test = create_test_dataset(clients_data, batch_size//2, n_x, n_y, sequence_length)

    # Résultats pour chaque cycle de communication
    communication_results = []
    global_params = None
    fedavg_params = None  # Pour garder une trace des paramètres FedAvg à chaque cycle
    global_kmeans_model = None
    param_shapes = None
    param_sizes = None

    # Pour le calcul de la bande passante
    traditional_bandwidth = []
    proposed_bandwidth = []

    # Itérer sur les cycles de communication
    for comm_round in range(n_communication_rounds):
        print(f"\n=== Cycle de communication {comm_round+1}/{n_communication_rounds} ===")

        # Phase 1: Premier cycle - FedAvg complet
        if comm_round == 0:
            print("Phase 1: Initialisation avec FedAvg")

            # Entraînement local sur chaque client
            parameters_by_client = []
            parameters_history_by_client = []

            for client_id, (X_c, Y_c) in enumerate(clients_data):
                print(f"Entraînement du client {client_id+1}/{n_clients}")
                params, history, _ = train_lstm(X_c, Y_c, n_a, n_x, n_y,
                                              num_epochs=n_epochs, seed=client_id)
                parameters_by_client.append(params)
                parameters_history_by_client.append(history)

            # FedAvg: Moyenne des paramètres des clients
            global_params = {}
            for key in parameters_by_client[0].keys():
                global_params[key] = np.mean([params[key] for params in parameters_by_client], axis=0)

            # Garder également ces paramètres comme référence FedAvg
            fedavg_params = copy.deepcopy(global_params)

            # Clusterisation globale des paramètres
            kmeans_models, cluster_labels, flat_params, param_shapes, param_sizes = cluster_parameters_by_epoch(
                parameters_history_by_client, n_clusters=n_clusters
            )

            # Stocker le dernier modèle KMeans comme modèle global
            global_kmeans_model = kmeans_models[-1]

            # Calcul de la bande passante pour la transmission complète des paramètres
            initial_bandwidth = calculate_transmission_size(global_params) * n_clients  # Total pour tous les clients
            traditional_bandwidth.append(initial_bandwidth)
            proposed_bandwidth.append(initial_bandwidth)  # Première phase identique

        # Phase 2: Cycles suivants - Mises à jour par matrices de transition
        else:
            print("Phase 2: Mise à jour avec matrices de transition")

            # MÉTHODE PROPOSÉE: Matrices de transition
            # Chaque client calcule sa matrice de transition
            client_transition_matrices = []

            # MÉTHODE TRADITIONNELLE: FedAvg standard
            # Entraînement local basé sur le modèle global FedAvg précédent
            fedavg_parameters_by_client = []

            for client_id, (X_c, Y_c) in enumerate(clients_data):
                print(f"Client {client_id+1}/{n_clients}")

                # POUR LES DEUX MÉTHODES: Entraînement local avec le dernier modèle global
                # Pour la méthode proposée, utiliser global_params
                # Pour FedAvg, utiliser fedavg_params

                # 1. Entraînement pour la méthode proposée
                params_proposed, history_proposed, _ = train_lstm(X_c, Y_c, n_a, n_x, n_y,
                                                  num_epochs=n_epochs,
                                                  seed=client_id+comm_round*100,
                                                  initial_params=global_params)

                # 2. Entraînement pour FedAvg traditionnel
                params_fedavg, _, _ = train_lstm(X_c, Y_c, n_a, n_x, n_y,
                                               num_epochs=n_epochs,
                                               seed=client_id+comm_round*100,
                                               initial_params=fedavg_params)
                fedavg_parameters_by_client.append(params_fedavg)

                # Calcul de la matrice de transition pour la méthode proposée
                flat_history = []
                for epoch_params in history_proposed:
                    flattened, _, _ = flatten_parameters(epoch_params)
                    flat_history.append(flattened)
                flat_history = np.array(flat_history)

                # Assigner des clusters à chaque état de paramètres
                cluster_assignments = global_kmeans_model.predict(flat_history)

                # Calculer la matrice de transition
                n_transitions = len(cluster_assignments) - 1
                transition_matrix = np.zeros((n_clusters, n_clusters))

                for t in range(n_transitions):
                    from_cluster = cluster_assignments[t]
                    to_cluster = cluster_assignments[t+1]
                    transition_matrix[from_cluster, to_cluster] += 1

                # Normaliser
                for i in range(n_clusters):
                    row_sum = np.sum(transition_matrix[i])
                    if row_sum > 0:
                        transition_matrix[i] = transition_matrix[i] / row_sum
                    else:
                        transition_matrix[i] = np.ones(n_clusters) / n_clusters

                client_transition_matrices.append(transition_matrix)

            # Mettre à jour le modèle global avec les matrices de transition
            global_params = update_with_transition_matrices(
                client_transition_matrices, global_kmeans_model,
                global_params, param_shapes, param_sizes
            )

            # Mettre à jour le modèle FedAvg traditionnel
            fedavg_params = {}
            for key in fedavg_parameters_by_client[0].keys():
                fedavg_params[key] = np.mean([params[key] for params in fedavg_parameters_by_client], axis=0)

            # Calcul de la bande passante
            # Pour FedAvg, tous les clients envoient tous leurs paramètres
            traditional_size = calculate_transmission_size(fedavg_params) * n_clients
            traditional_bandwidth.append(traditional_size)

            # Pour la méthode proposée, seulement les matrices de transition
            transition_size = n_clusters * n_clusters * 4 * n_clients  # Taille en octets (4 octets par float)
            proposed_bandwidth.append(transition_size)

        # Évaluation des deux modèles pour ce cycle
        proposed_acc, proposed_loss = evaluate_lstm(X_test, Y_test, global_params)
        fedavg_acc, fedavg_loss = evaluate_lstm(X_test, Y_test, fedavg_params)

        # Stocker les résultats
        cycle_result = {
            "cycle": comm_round,
            "proposed_accuracy": proposed_acc,
            "proposed_loss": proposed_loss,
            "fedavg_accuracy": fedavg_acc,
            "fedavg_loss": fedavg_loss,
            "traditional_bandwidth": traditional_bandwidth[-1],
            "proposed_bandwidth": proposed_bandwidth[-1]
        }
        communication_results.append(cycle_result)

        print(f"Résultats du cycle {comm_round+1}:")
        print(f"  Méthode proposée - Accuracy: {proposed_acc:.4f}, Loss: {proposed_loss:.4f}")
        print(f"  FedAvg traditionnel - Accuracy: {fedavg_acc:.4f}, Loss: {fedavg_loss:.4f}")
        print(f"  Bande passante traditionnelle: {traditional_bandwidth[-1]/1024:.2f} KB")
        print(f"  Bande passante proposée: {proposed_bandwidth[-1]/1024:.2f} KB")
        if comm_round > 0:
            saving = (1 - proposed_bandwidth[-1]/traditional_bandwidth[-1]) * 100
            print(f"  Économie de bande passante: {saving:.2f}%")

    # Calcul du temps total d'exécution
    total_time = time.time() - start_time

    # Visualisation des résultats
    visualize_communication_rounds_comparison(communication_results)

    # Résumé des résultats
    print("\n=== Résumé de l'expérience en deux phases ===")
    print(f"Nombre de cycles de communication: {n_communication_rounds}")
    print(f"Temps total d'exécution: {total_time:.2f} secondes")

    # Calcul des économies de bande passante cumulées
    total_traditional = sum(traditional_bandwidth)
    total_proposed = sum(proposed_bandwidth)
    total_saving = (1 - total_proposed/total_traditional) * 100

    print(f"Bande passante traditionnelle totale: {total_traditional/1024:.2f} KB")
    print(f"Bande passante proposée totale: {total_proposed/1024:.2f} KB")
    print(f"Économie de bande passante totale: {total_saving:.2f}%")

    return communication_results, global_params, fedavg_params


def cluster_trajectories_with_existing_centers(client_seeds_history, cluster_centers):
    """
    Attribue des clusters aux trajectoires en utilisant des centres préexistants.

    Arguments:
    client_seeds_history -- liste d'historiques de paramètres pour différentes seeds
    cluster_centers -- centres des clusters préétablis

    Returns:
    flat_labels -- étiquettes de cluster pour chaque seed à chaque époque
    """
    n_seeds = len(client_seeds_history)
    n_epochs = len(client_seeds_history[0])
    n_clusters = len(cluster_centers)

    # Aplatir les paramètres pour toutes les graines et époques
    flat_labels = np.zeros((n_seeds, n_epochs), dtype=int)

    for seed in range(n_seeds):
        for epoch in range(n_epochs):
            # Aplatir les paramètres de cette époque
            flattened, _, _ = flatten_parameters(client_seeds_history[seed][epoch])

            # Calculer les distances aux centres des clusters
            distances = np.array([np.linalg.norm(flattened - center) for center in cluster_centers])

            # Attribuer au cluster le plus proche
            flat_labels[seed, epoch] = np.argmin(distances)

    return flat_labels

def compute_transition_matrix_from_labels(flat_labels, n_clusters):
    """
    Calcule la matrice de transition à partir des séquences de labels de clusters.

    Arguments:
    flat_labels -- étiquettes de cluster pour chaque seed à chaque époque
    n_clusters -- nombre de clusters

    Returns:
    transition_matrix -- matrice de transition de Markov
    """
    n_seeds, n_epochs = flat_labels.shape

    transition_counts = np.zeros((n_clusters, n_clusters))

    # Compter les transitions
    for seed in range(n_seeds):
        for epoch in range(n_epochs - 1):
            from_cluster = flat_labels[seed, epoch]
            to_cluster = flat_labels[seed, epoch + 1]
            transition_counts[from_cluster, to_cluster] += 1

    # Normaliser pour obtenir les probabilités
    transition_matrix = np.zeros_like(transition_counts)
    for i in range(n_clusters):
        row_sum = np.sum(transition_counts[i])
        if row_sum > 0:
            transition_matrix[i] = transition_counts[i] / row_sum
        else:
            # Si aucune transition n'est observée depuis ce cluster, distribution uniforme
            transition_matrix[i] = 1.0 / n_clusters

    return transition_matrix

def update_with_transition_matrices_only(client_transition_matrices, client_cluster_centers, param_shapes, param_sizes, n_clients):
    """
    Met à jour le modèle global en utilisant uniquement les matrices de transition.

    Arguments:
    client_transition_matrices -- liste des matrices de transition de chaque client
    client_cluster_centers -- liste des centres de clusters de chaque client (définis en phase 1)
    param_shapes -- formes des paramètres
    param_sizes -- tailles des paramètres aplatis
    n_clients -- nombre de clients

    Returns:
    updated_global_params -- paramètres globaux mis à jour
    """
    # Initialiser les paramètres globaux
    aggregated_params_flat = np.zeros_like(client_cluster_centers[0][0])

    # Pour chaque client
    for client_id in range(n_clients):
        transition_matrix = client_transition_matrices[client_id]
        cluster_centers = client_cluster_centers[client_id]
        n_clusters = transition_matrix.shape[0]

        # Calculer la distribution stationnaire de la matrice de transition
        pi = np.ones(n_clusters) / n_clusters  # Distribution initiale uniforme
        for _ in range(100):  # Nombre d'itérations arbitraire pour convergence
            pi_new = np.dot(pi, transition_matrix)
            if np.allclose(pi, pi_new):
                break
            pi = pi_new

        # Pondérer les centres par la distribution stationnaire
        client_params_flat = np.zeros_like(cluster_centers[0])
        for i in range(n_clusters):
            client_params_flat += pi[i] * cluster_centers[i]

        # Ajouter à l'agrégation globale
        aggregated_params_flat += client_params_flat / n_clients

    # Reconstruire les paramètres à leur forme d'origine
    updated_global_params = unflatten_parameters(aggregated_params_flat, param_shapes, param_sizes)

    return updated_global_params

def federated_main_two_phase_local(
    n_clients=3, n_epochs=50, n_clusters=3,
    n_a=64, n_x=10, n_y=5,
    batch_size=32, sequence_length=10,
    num_transfer_epochs=5, learning_rate=0.01,
    n_communication_rounds=6,  # Nombre total de cycles de communication
    use_synthetic_data=True
):
    """
    Pipeline complet de Federated Learning avec approche en deux phases et clusterisation locale:
    Phase 1: Initialisation avec FedAvg + centres de clusters
    Phase 2: Mises à jour avec matrices de transition uniquement
    Compare aussi avec la méthode traditionnelle FedAvg à chaque cycle
    """
    print("=== Lancement du Federated Learning en deux phases avec clusterisation locale ===")
    start_time = time.time()

    # Génération/chargement des données comme avant
    if use_synthetic_data:
        print("Génération des données synthétiques...")
        clients_data = []
        for seed in range(n_clients):
            X_train = np.random.randn(n_x, batch_size, sequence_length)
            Y_train = np.zeros((n_y, batch_size, sequence_length))
            for t in range(sequence_length):
                for i in range(batch_size):
                    class_idx = np.random.randint(0, n_y)
                    Y_train[class_idx, i, t] = 1
            clients_data.append((X_train, Y_train))

        # Données de test
        X_test = np.random.randn(n_x, batch_size//2, sequence_length)
        Y_test = np.zeros((n_y, batch_size//2, sequence_length))
        for t in range(sequence_length):
            for i in range(batch_size//2):
                class_idx = np.random.randint(0, n_y)
                Y_test[class_idx, i, t] = 1
    else:
        # Code pour les données réelles
        print("Chargement des données réelles...")
        clients_data = load_real_data(n_clients, 1, n_x, n_y, sequence_length)
        # Extraire seulement la première seed pour chaque client
        clients_data = [client_seeds[0] for client_seeds in clients_data]
        # Créer des données de test
        X_test, Y_test = create_test_dataset(clients_data, batch_size//2, n_x, n_y, sequence_length)

    # Résultats pour chaque cycle de communication
    communication_results = []
    global_params = None
    fedavg_params = None  # Pour garder une trace des paramètres FedAvg à chaque cycle
    client_cluster_centers = []  # Stockage des centres de clusters pour chaque client
    param_shapes = None
    param_sizes = None

    # Pour le calcul de la bande passante
    traditional_bandwidth = []
    proposed_bandwidth = []

    # Itérer sur les cycles de communication
    for comm_round in range(n_communication_rounds):
        print(f"\n=== Cycle de communication {comm_round+1}/{n_communication_rounds} ===")

        # Phase 1: Premier cycle - FedAvg complet et centres de clusters
        if comm_round == 0:
            print("Phase 1: Initialisation avec FedAvg et création des centres de clusters")

            # Entraînement local sur chaque client
            parameters_by_client = []
            parameters_history_by_client = []

            for client_id, (X_c, Y_c) in enumerate(clients_data):
                print(f"Entraînement du client {client_id+1}/{n_clients}")

                # Plusieurs seeds pour chaque client (pour la clusterisation locale)
                client_seeds_history = []
                for seed_id in range(5):  # Utiliser 5 seeds différentes par client
                    base_seed = client_id * 100 + seed_id
                    params, history, _ = train_lstm(X_c, Y_c, n_a, n_x, n_y,
                                                  num_epochs=n_epochs, seed=base_seed)
                    client_seeds_history.append(history)

                # Stocker la dernière itération des paramètres pour FedAvg initial
                parameters_by_client.append(client_seeds_history[0][-1])
                parameters_history_by_client.append(client_seeds_history)

            # FedAvg: Moyenne des paramètres des clients
            global_params = {}
            for key in parameters_by_client[0].keys():
                global_params[key] = np.mean([params[key] for params in parameters_by_client], axis=0)

            # Garder également ces paramètres comme référence FedAvg
            fedavg_params = copy.deepcopy(global_params)

            # Réaliser la clusterisation locale au niveau de chaque client
            client_results = []

            for client_id in range(n_clients):
                # Clusteriser les paramètres de ce client
                cluster_centers, transition_matrix = cluster_local_by_client_single(
                    parameters_history_by_client[client_id], n_clusters=n_clusters
                )

                # Conserver les formes et tailles des paramètres (identiques pour tous les clients)
                if client_id == 0:
                    _, param_shapes, param_sizes = flatten_parameters(parameters_history_by_client[0][0][0])

                # Stocker les centres et matrices pour chaque client
                client_results.append((cluster_centers, transition_matrix))

                # Stocker uniquement les centres pour utilisation future
                client_cluster_centers.append(cluster_centers)

            # Calcul de la bande passante
            # Pour la phase 1, les deux approches envoient les paramètres complets
            initial_bandwidth = calculate_transmission_size(global_params) * n_clients

            # Pour la méthode proposée, on envoie aussi les clusters
            clusters_bandwidth = calculate_transmission_size(parameters=None, client_centers=client_results)
            proposed_initial_bandwidth = initial_bandwidth + clusters_bandwidth

            traditional_bandwidth.append(initial_bandwidth)
            proposed_bandwidth.append(proposed_initial_bandwidth)

        # Phase 2: Cycles suivants - Mises à jour par matrices de transition uniquement
        else:
            print("Phase 2: Mise à jour avec matrices de transition uniquement")

            # MÉTHODE PROPOSÉE: Matrices de transition uniquement
            client_transition_matrices = []

            # MÉTHODE TRADITIONNELLE: FedAvg standard
            fedavg_parameters_by_client = []

            for client_id, (X_c, Y_c) in enumerate(clients_data):
                print(f"Client {client_id+1}/{n_clients}")

                # 1. Entraînement pour FedAvg traditionnel
                params_fedavg, _, _ = train_lstm(X_c, Y_c, n_a, n_x, n_y,
                                               num_epochs=n_epochs,
                                               seed=client_id+comm_round*100,
                                               initial_params=fedavg_params)
                fedavg_parameters_by_client.append(params_fedavg)

                # 2. Pour la méthode proposée: générer plusieurs séquences d'entraînement
                client_seeds_history = []
                for seed_id in range(5):  # Utiliser 5 seeds différentes
                    base_seed = client_id * 100 + seed_id + comm_round * 1000
                    params, history, _ = train_lstm(X_c, Y_c, n_a, n_x, n_y,
                                                  num_epochs=n_epochs,
                                                  seed=base_seed,
                                                  initial_params=global_params)
                    client_seeds_history.append(history)

                # Transformer les trajectoires d'entraînement en séquences de clusters
                # en utilisant les centres de la Phase 1
                flat_labels = cluster_trajectories_with_existing_centers(
                    client_seeds_history, client_cluster_centers[client_id]
                )

                # Calculer uniquement la matrice de transition à partir de ces labels
                transition_matrix = compute_transition_matrix_from_labels(flat_labels, n_clusters)

                # Stocker la matrice de transition pour ce client
                client_transition_matrices.append(transition_matrix)

            # Mettre à jour le modèle global en utilisant uniquement les matrices de transition
            global_params = update_with_transition_matrices_only(
                client_transition_matrices, client_cluster_centers, param_shapes, param_sizes, n_clients
            )

            # Mettre à jour le modèle FedAvg traditionnel
            fedavg_params = {}
            for key in fedavg_parameters_by_client[0].keys():
                fedavg_params[key] = np.mean([params[key] for params in fedavg_parameters_by_client], axis=0)

            # Calcul de la bande passante
            # Pour FedAvg, tous les clients envoient tous leurs paramètres
            traditional_size = calculate_transmission_size(fedavg_params) * n_clients
            traditional_bandwidth.append(traditional_size)

            # Pour la méthode proposée, seulement les matrices de transition
            # Taille d'une matrice de transition: n_clusters x n_clusters x 4 octets
            proposed_size = n_clusters * n_clusters * 4 * n_clients
            proposed_bandwidth.append(proposed_size)

        # Évaluation des deux modèles pour ce cycle
        proposed_acc, proposed_loss = evaluate_lstm(X_test, Y_test, global_params)
        fedavg_acc, fedavg_loss = evaluate_lstm(X_test, Y_test, fedavg_params)

        # Stockage et affichage des résultats comme avant
        cycle_result = {
            "cycle": comm_round,
            "proposed_accuracy": proposed_acc,
            "proposed_loss": proposed_loss,
            "fedavg_accuracy": fedavg_acc,
            "fedavg_loss": fedavg_loss,
            "traditional_bandwidth": traditional_bandwidth[-1],
            "proposed_bandwidth": proposed_bandwidth[-1]
        }
        communication_results.append(cycle_result)

        print(f"Résultats du cycle {comm_round+1}:")
        print(f"  Méthode proposée - Accuracy: {proposed_acc:.4f}, Loss: {proposed_loss:.4f}")
        print(f"  FedAvg traditionnel - Accuracy: {fedavg_acc:.4f}, Loss: {fedavg_loss:.4f}")
        print(f"  Bande passante traditionnelle: {traditional_bandwidth[-1]/1024:.2f} KB")
        print(f"  Bande passante proposée: {proposed_bandwidth[-1]/1024:.2f} KB")
        if comm_round > 0:
            saving = (1 - proposed_bandwidth[-1]/traditional_bandwidth[-1]) * 100
            print(f"  Économie de bande passante: {saving:.2f}%")

    # Calcul du temps total d'exécution et affichage des résultats comme avant
    total_time = time.time() - start_time

    # Visualisation des résultats
    visualize_communication_rounds_comparison(communication_results)

    # Résumé des résultats
    print("\n=== Résumé de l'expérience en deux phases avec clusterisation locale ===")
    print(f"Nombre de cycles de communication: {n_communication_rounds}")
    print(f"Temps total d'exécution: {total_time:.2f} secondes")

    # Calcul des économies de bande passante cumulées
    total_traditional = sum(traditional_bandwidth)
    total_proposed = sum(proposed_bandwidth)
    total_saving = (1 - total_proposed/total_traditional) * 100

    print(f"Bande passante traditionnelle totale: {total_traditional/1024:.2f} KB")
    print(f"Bande passante proposée totale: {total_proposed/1024:.2f} KB")
    print(f"Économie de bande passante totale: {total_saving:.2f}%")

    return communication_results, global_params, fedavg_params

def cluster_local_by_client_single(client_seeds_history, n_clusters):
    """
    Version simplifiée de cluster_local_by_client pour un seul client

    Arguments:
    client_seeds_history -- liste de listes de dictionnaires Python contenant les paramètres
    n_clusters -- nombre de clusters à former

    Returns:
    cluster_centers -- centres des clusters
    transition_matrix -- matrice de transition
    """
    n_seeds = len(client_seeds_history)
    n_epochs = len(client_seeds_history[0])

    # Obtenir les formes et tailles des paramètres
    _, param_shapes, param_sizes = flatten_parameters(client_seeds_history[0][0])

    # Aplatir les paramètres pour toutes les graines et époques
    flat_params = []
    for seed in range(n_seeds):
        seed_params = []
        for epoch in range(n_epochs):
            flattened, _, _ = flatten_parameters(client_seeds_history[seed][epoch])
            seed_params.append(flattened)
        flat_params.append(seed_params)

    flat_params = np.array(flat_params)

    # Clusteriser par époque
    kmeans_models = []
    cluster_labels = np.zeros((n_seeds, n_epochs), dtype=int)

    for epoch in range(n_epochs):
        epoch_params = flat_params[:, epoch, :]
        kmeans = KMeans(n_clusters=min(n_clusters, n_seeds), random_state=42)
        cluster_labels[:, epoch] = kmeans.fit_predict(epoch_params)
        kmeans_models.append(kmeans)

    # Calculer la matrice de transition
    transition_matrix = np.zeros((n_clusters, n_clusters))

    # Compter les transitions
    for seed in range(n_seeds):
        for epoch in range(n_epochs - 1):
            from_cluster = cluster_labels[seed, epoch]
            to_cluster = cluster_labels[seed, epoch + 1]
            transition_matrix[from_cluster, to_cluster] += 1

    # Normaliser pour obtenir les probabilités
    for i in range(n_clusters):
        row_sum = np.sum(transition_matrix[i])
        if row_sum > 0:
            transition_matrix[i] = transition_matrix[i] / row_sum
        else:
            # Si aucune transition n'est observée depuis ce cluster, distribution uniforme
            transition_matrix[i] = 1.0 / n_clusters

    # Utiliser les centres de clusters du dernier modèle KMeans
    last_kmeans = kmeans_models[-1]
    cluster_centers = last_kmeans.cluster_centers_

    return cluster_centers, transition_matrix

def analyze_computational_complexity(n_clients, n_a, n_x, n_y, n_epochs, n_clusters, sequence_length, batch_size):
    """
    Analyse la complexité computationnelle du processus d'apprentissage fédéré avec compression markovienne.
    """
    complexity_dict = {}

    # 1. Complexité de l'entraînement local sur un client (par époque)
    # Forward pass LSTM: O(batch_size * sequence_length * (n_a^2 + n_a*n_x + n_a*n_y))
    # Backward pass LSTM: O(batch_size * sequence_length * (n_a^2 + n_a*n_x + n_a*n_y))
    forward_complexity = batch_size * sequence_length * (n_a**2 + n_a*n_x + n_a*n_y)
    backward_complexity = batch_size * sequence_length * (n_a**2 + n_a*n_x + n_a*n_y)
    client_training_complexity = n_epochs * (forward_complexity + backward_complexity)
    complexity_dict["Local Training (per client)"] = client_training_complexity

    # 2. Complexité de la clusterisation
    # K-means: O(n_clients * n_epochs * n_clusters * d * i)
    # où d est la dimension des paramètres et i est le nombre d'itérations K-means
    d = n_a**2 + n_a*n_x + n_a*n_y + n_a + n_y*n_a + n_y
    kmeans_iterations = 100  # Hypothèse pour le nombre d'itérations K-means
    clustering_complexity = n_clients * n_epochs * n_clusters * d * kmeans_iterations
    complexity_dict["Clustering"] = clustering_complexity

    # 3. Complexité du calcul de la matrice de transition
    # O(n_clients * n_epochs * n_clusters^2)
    transition_matrix_complexity = n_clients * n_epochs * n_clusters**2
    complexity_dict["Transition Matrix Computation"] = transition_matrix_complexity

    # 4. Complexité de la simulation de trajectoire
    # O(n_steps * n_clusters)
    trajectory_complexity = n_epochs * n_clusters
    complexity_dict["Trajectory Simulation"] = trajectory_complexity

    # 5. Complexité de la méthode traditionnelle (moyenne des paramètres)
    # O(n_clients * d)
    traditional_complexity = n_clients * d
    complexity_dict["Traditional Aggregation"] = traditional_complexity

    # 6. Complexité du transfer learning
    # Similaire à l'entraînement local mais avec moins d'époques
    transfer_epochs = 5  # Hypothèse
    transfer_complexity = transfer_epochs * (forward_complexity + backward_complexity)
    complexity_dict["Transfer Learning"] = transfer_complexity

    # Complexité totale de l'approche proposée vs approche traditionnelle
    proposed_approach = (n_clients * client_training_complexity +
                         clustering_complexity +
                         transition_matrix_complexity +
                         trajectory_complexity +
                         transfer_complexity)

    traditional_approach = n_clients * client_training_complexity + traditional_complexity

    complexity_dict["Total (Proposed Approach)"] = proposed_approach
    complexity_dict["Total (Traditional Approach)"] = traditional_approach
    complexity_dict["Efficiency Ratio"] = traditional_approach / proposed_approach

    return complexity_dict

def visualize_computational_complexity(complexity_dict):
    """
    Visualise la complexité computationnelle.
    """
    # Extraction des composantes principales pour la visualisation
    components = ["Local Training (per client)", "Clustering", "Transition Matrix Computation",
                 "Trajectory Simulation", "Transfer Learning"]
    values = [complexity_dict[comp] for comp in components]

    # Normalisation pour une meilleure visualisation
    normalized_values = np.array(values) / np.sum(values) * 100

    # Création du graphique
    plt.figure(figsize=(12, 6))
    bars = plt.bar(components, normalized_values, color=['blue', 'green', 'orange', 'red', 'purple'])

    # Ajout des valeurs en pourcentage
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                 f'{height:.1f}%', ha='center', va='bottom')

    plt.title('Décomposition de la complexité computationnelle')
    plt.ylabel('Pourcentage du temps de calcul total')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('decomposition_complexite.png')
    plt.show()

    # Comparaison des approches
    plt.figure(figsize=(10, 6))
    ratio = complexity_dict["Efficiency Ratio"]
    plt.bar(['Approche traditionnelle', 'Approche proposée'],
            [100, 100/ratio],
            color=['gray', 'green'])
    plt.title(f'Comparaison de l\'efficacité (Traditionnel / Proposé = {ratio:.2f}x)')
    plt.ylabel('Complexité relative (%)')
    plt.ylim(0, max(100, 100/ratio) * 1.1)
    plt.tight_layout()
    plt.savefig('comparaison_complexite.png')
    plt.show()

def visualize_results(original_acc, compressed_acc, transfer_acc, full_params_acc, original_loss, compressed_loss, transfer_loss, full_params_loss):
    """
    Visualise les résultats de l'expérience avec comparaison des méthodes.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Graphique des précisions
    labels = ['Original', 'Compressé', 'Transfert', 'Full Params']
    accuracies = [original_acc, compressed_acc, transfer_acc, full_params_acc]
    colors = ['blue', 'orange', 'green', 'red']
    ax1.bar(labels, accuracies, color=colors)
    ax1.set_ylabel('Précision')
    ax1.set_title('Comparaison des précisions')

    # Graphique des pertes
    losses = [original_loss, compressed_loss, transfer_loss, full_params_loss]
    ax2.bar(labels, losses, color=colors)
    ax2.set_ylabel('Perte')
    ax2.set_title('Comparaison des pertes')

    plt.tight_layout()
    plt.savefig('resultats_compression_lstm.png')
    plt.show()

def visualize_transition_matrix(transition_matrix):
    """
    Visualise la matrice de transition.
    """
    plt.figure(figsize=(10, 8))
    plt.imshow(transition_matrix, cmap='viridis', interpolation='none')
    plt.colorbar(label='Probabilité de transition')
    plt.title('Matrice de transition de Markov')
    plt.xlabel('Cluster de destination')
    plt.ylabel('Cluster de départ')

    # Ajouter les valeurs sur la figure
    for i in range(transition_matrix.shape[0]):
        for j in range(transition_matrix.shape[1]):
            plt.text(j, i, f'{transition_matrix[i, j]:.2f}',
                     ha='center', va='center',
                     color='white' if transition_matrix[i, j] > 0.5 else 'black')

    plt.tight_layout()
    plt.savefig('matrice_transition_markov.png')
    plt.show()

def compress_parameters(parameters_history_by_seed, n_clusters=3, n_steps=None):
    """
    Compresse les trajectoires de paramètres en utilisant des clusters et des processus de Markov.
    """
    n_seeds = len(parameters_history_by_seed)
    n_epochs = len(parameters_history_by_seed[0])

    if n_steps is None:
        n_steps = n_epochs

    # Clusteriser les paramètres
    kmeans_models, cluster_labels, flat_params, param_shapes, param_sizes = cluster_parameters_by_epoch(
        parameters_history_by_seed, n_clusters)

    # Calculer la matrice de transition
    transition_matrix = compute_transition_matrix(cluster_labels)

    # Simuler de nouvelles trajectoires
    compressed_params_flat = []

    for seed in range(n_seeds):
        # Utiliser le premier cluster observé pour cette graine comme point de départ
        initial_cluster = cluster_labels[seed, 0]
        trajectory, _ = simulate_parameter_trajectory(initial_cluster, transition_matrix, n_steps, kmeans_models)
        compressed_params_flat.append(trajectory[-1])  # Utiliser le dernier état simulé

    # Calculer les paramètres moyens compressés (moyennant sur les graines)
    avg_compressed_params_flat = np.mean(compressed_params_flat, axis=0)
    compressed_params = unflatten_parameters(avg_compressed_params_flat, param_shapes, param_sizes)

    # Paramètres originaux moyens pour comparaison
    avg_original_params_flat = np.mean(flat_params[:, -1, :], axis=0)  # Moyenne de la dernière époque
    original_params = unflatten_parameters(avg_original_params_flat, param_shapes, param_sizes)

    return compressed_params, original_params, transition_matrix, kmeans_models, cluster_labels, flat_params, param_shapes, param_sizes

def compare_approaches(results_global, results_local):
    """
    Compare et visualise les résultats des deux approches.

    Arguments:
    results_global -- résultats de l'approche globale
    results_local -- résultats de l'approche locale
    """
    # Fonction utilitaire pour récupérer en toute sécurité les valeurs des dictionnaires
    def safe_get(results, key, default=0):
        value = results.get(key, default)
        if not isinstance(value, (int, float)) or not np.isfinite(value):
            return default
        return value

    print("\n\n" + "=" * 80)
    print("COMPARAISON DES APPROCHES")
    print("=" * 80)

    # 1. Comparaison des performances (précision)
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # Précision
    acc_labels = ['Original', 'Compressé', 'Après transfert']
    acc_global = [results_global['original_acc'], results_global['compressed_acc'], results_global['transfer_acc']]
    acc_local = [results_local['original_acc'], results_local['local_compressed_acc'], results_local['local_transfer_acc']]

    x = np.arange(len(acc_labels))
    width = 0.35

    axes[0, 0].bar(x - width/2, acc_global, width, label='Approche globale')
    axes[0, 0].bar(x + width/2, acc_local, width, label='Approche locale')
    axes[0, 0].set_ylabel('Précision')
    axes[0, 0].set_title('Comparaison des précisions')
    axes[0, 0].set_xticks(x)
    axes[0, 0].set_xticklabels(acc_labels)
    axes[0, 0].legend()

    # Ajouter les valeurs sur les barres
    for i, v in enumerate(acc_global):
        axes[0, 0].text(i - width/2, v + 0.01, f"{v:.3f}", ha='center')
    for i, v in enumerate(acc_local):
        axes[0, 0].text(i + width/2, v + 0.01, f"{v:.3f}", ha='center')

    # Perte
    loss_labels = ['Original', 'Compressé', 'Après transfert']
    loss_global = [results_global['original_loss'], results_global['compressed_loss'], results_global['transfer_loss']]
    loss_local = [results_local['original_loss'], results_local['local_compressed_loss'], results_local['local_transfer_loss']]

    axes[0, 1].bar(x - width/2, loss_global, width, label='Approche globale')
    axes[0, 1].bar(x + width/2, loss_local, width, label='Approche locale')
    axes[0, 1].set_ylabel('Perte')
    axes[0, 1].set_title('Comparaison des pertes')
    axes[0, 1].set_xticks(x)
    axes[0, 1].set_xticklabels(loss_labels)
    axes[0, 1].legend()

    # Ajouter les valeurs sur les barres
    for i, v in enumerate(loss_global):
        axes[0, 1].text(i - width/2, v + 0.01, f"{v:.3f}", ha='center')
    for i, v in enumerate(loss_local):
        axes[0, 1].text(i + width/2, v + 0.01, f"{v:.3f}", ha='center')

    # 2. Comparaison des tailles de transmission
    size_labels = ['Original', 'Compressé']
    size_global = [results_global['original_size']/1024, results_global['compressed_size']/1024] # en KB
    size_local = [results_local['original_size']/1024, results_local['local_compressed_size']/1024] # en KB

    axes[1, 0].bar(x[:2] - width/2, size_global, width, label='Approche globale')
    axes[1, 0].bar(x[:2] + width/2, size_local, width, label='Approche locale')
    axes[1, 0].set_ylabel('Taille (KB)')
    axes[1, 0].set_title('Comparaison des tailles de transmission')
    axes[1, 0].set_xticks(x[:2])
    axes[1, 0].set_xticklabels(size_labels)
    axes[1, 0].legend()

    # Ajouter les valeurs sur les barres
    for i, v in enumerate(size_global):
        axes[1, 0].text(i - width/2, v + 0.5, f"{v:.1f}", ha='center')
    for i, v in enumerate(size_local):
        axes[1, 0].text(i + width/2, v + 0.5, f"{v:.1f}", ha='center')

    # 3. Comparaison des temps d'exécution
    time_labels = ['Entraînement', 'Compression', 'Transfer']
    time_global = [results_global['local_training_time'], results_global['compression_time'], results_global['transfer_time']]
    time_local = [results_local['local_training_time'], results_local['clustering_time'] + results_local['aggregation_time'], results_local['transfer_time']]

    axes[1, 1].bar(x[:3] - width/2, time_global, width, label='Approche globale')
    axes[1, 1].bar(x[:3] + width/2, time_local, width, label='Approche locale')
    axes[1, 1].set_ylabel('Temps (secondes)')
    axes[1, 1].set_title('Comparaison des temps d\'exécution')
    axes[1, 1].set_xticks(x[:3])
    axes[1, 1].set_xticklabels(time_labels)
    axes[1, 1].legend()

    # Ajouter les valeurs sur les barres
    for i, v in enumerate(time_global):
        axes[1, 1].text(i - width/2, v + 0.5, f"{v:.1f}", ha='center')
    for i, v in enumerate(time_local):
        axes[1, 1].text(i + width/2, v + 0.5, f"{v:.1f}", ha='center')

    plt.tight_layout()
    plt.savefig('comparaison_approches.png')
    plt.show()

    # Tableau récapitulatif
    print("\nRÉSUMÉ COMPARATIF :")
    print("-" * 80)
    print(f"{'Métrique':<30} | {'Approche globale':<20} | {'Approche locale':<20}")
    print("-" * 80)

    # Vérification des valeurs avant affichage pour éviter les erreurs de formatage
    # dues à des valeurs NaN ou inf qui résulteraient de divisions par zéro

    # Taux de compression
    global_compression = safe_get(results_global, 'compression_ratio', 0)
    local_compression = safe_get(results_local, 'compression_ratio', 0)
    if not isinstance(global_compression, (int, float)) or not np.isfinite(global_compression):
        global_compression = 0
    if not isinstance(local_compression, (int, float)) or not np.isfinite(local_compression):
        local_compression = 0
    print(f"{'Taux de compression':<30} | {global_compression:<20.2f} | {local_compression:<20.2f}")

    # Économie de bande passante
    global_saving = safe_get(results_global, 'bandwidth_saving', 0)
    local_saving = safe_get(results_local, 'bandwidth_saving', 0)
    if not isinstance(global_saving, (int, float)) or not np.isfinite(global_saving):
        global_saving = 0
    if not isinstance(local_saving, (int, float)) or not np.isfinite(local_saving):
        local_saving = 0
    print(f"{'Économie de bande passante (%)':<30} | {global_saving:<20.2f} | {local_saving:<20.2f}")

    # Perte relative de performance
    global_loss = safe_get(results_global, 'performance_loss', 0)
    local_loss = safe_get(results_local, 'performance_loss', 0)
    if not isinstance(global_loss, (int, float)) or not np.isfinite(global_loss):
        global_loss = 0
    if not isinstance(local_loss, (int, float)) or not np.isfinite(local_loss):
        local_loss = 0
    print(f"{'Perte relative de perf. (%)':<30} | {global_loss:<20.2f} | {local_loss:<20.2f}")

    # Récupération après transfert
    global_recovery = safe_get(results_global, 'performance_recovery', 0)
    local_recovery = safe_get(results_local, 'performance_recovery', 0)
    if not isinstance(global_recovery, (int, float)) or not np.isfinite(global_recovery):
        global_recovery = 0
    if not isinstance(local_recovery, (int, float)) or not np.isfinite(local_recovery):
        local_recovery = 0
    print(f"{'Récupération après transfert (%)':<30} | {global_recovery:<20.2f} | {local_recovery:<20.2f}")

    # Efficacité computationnelle
    global_efficiency = safe_get(results_global, 'computational_efficiency', 0)
    local_efficiency = safe_get(results_local, 'computational_efficiency', 0)
    if not isinstance(global_efficiency, (int, float)) or not np.isfinite(global_efficiency):
        global_efficiency = 0
    if not isinstance(local_efficiency, (int, float)) or not np.isfinite(local_efficiency):
        local_efficiency = 0
    print(f"{'Efficacité computationnelle':<30} | {global_efficiency:<20.2f} | {local_efficiency:<20.2f}")

    # Temps d'exécution
    global_time = safe_get(results_global, 'total_time', 0)
    local_time = safe_get(results_local, 'total_time', 0)
    if not isinstance(global_time, (int, float)) or not np.isfinite(global_time):
        global_time = 0
    if not isinstance(local_time, (int, float)) or not np.isfinite(local_time):
        local_time = 0
    print(f"{'Temps total d exécution (s)':<30} | {global_time:<20.2f} | {local_time:<20.2f}")

    print("-" * 80)

    # Analyse des résultats
    print("\nANALYSE DES RÉSULTATS :")
    print("-" * 80)

    # Récupérer les valeurs avec une gestion sécurisée
    def safe_compare(local_val, global_val, better_if_higher=True):
        # Vérifie si les deux valeurs sont des nombres valides
        if not isinstance(local_val, (int, float)) or not np.isfinite(local_val) or \
           not isinstance(global_val, (int, float)) or not np.isfinite(global_val):
            return None

        # Si les deux valeurs sont 0, on ne peut pas vraiment comparer
        if local_val == 0 and global_val == 0:
            return None

        # Comparaison selon le critère spécifié
        if better_if_higher:
            return local_val > global_val
        else:
            return local_val < global_val

    # Compression (plus élevé = mieux)
    comp_result = safe_compare(
        safe_get(results_local, 'compression_ratio'),
        safe_get(results_global, 'compression_ratio'),
        True
    )
    if comp_result is not None:
        if comp_result:
            diff = safe_get(results_local, 'compression_ratio') - safe_get(results_global, 'compression_ratio')
            print(f"✓ L'approche locale offre un meilleur taux de compression (+{diff:.2f}x)")
        else:
            diff = safe_get(results_global, 'compression_ratio') - safe_get(results_local, 'compression_ratio')
            print(f"✗ L'approche locale offre un moins bon taux de compression (-{diff:.2f}x)")
    else:
        print("⚠ Impossible de comparer les taux de compression (valeurs non valides ou nulles)")

    # Performance
    perf_result = safe_compare(
        safe_get(results_local, 'performance_loss'),
        safe_get(results_global, 'performance_loss'),
        False
    )
    if perf_result is not None:
        if perf_result:
            diff = safe_get(results_global, 'performance_loss') - safe_get(results_local, 'performance_loss')
            print(f"✓ L'approche locale préserve mieux les performances (-{diff:.2f}% de perte)")
        else:
            diff = safe_get(results_local, 'performance_loss') - safe_get(results_global, 'performance_loss')
            print(f"✗ L'approche locale préserve moins bien les performances (+{diff:.2f}% de perte)")
    else:
        print("⚠ Impossible de comparer les pertes de performance (valeurs égales ou non valides)")

    # Récupération
    recovery_result = safe_compare(
        safe_get(results_local, 'performance_recovery'),
        safe_get(results_global, 'performance_recovery'),
        True
    )
    if recovery_result is not None:
        if recovery_result:
            diff = safe_get(results_local, 'performance_recovery') - safe_get(results_global, 'performance_recovery')
            print(f"✓ L'approche locale montre une meilleure récupération après transfert (+{diff:.2f}%)")
        else:
            diff = safe_get(results_global, 'performance_recovery') - safe_get(results_local, 'performance_recovery')
            print(f"✗ L'approche locale montre une moins bonne récupération après transfert (-{diff:.2f}%)")
    else:
        print("⚠ Impossible de comparer la récupération après transfert (valeurs égales ou non valides)")

    # Efficacité
    efficiency_result = safe_compare(
        safe_get(results_local, 'computational_efficiency'),
        safe_get(results_global, 'computational_efficiency'),
        True
    )
    if efficiency_result is not None:
        if efficiency_result:
            diff = safe_get(results_local, 'computational_efficiency') - safe_get(results_global, 'computational_efficiency')
            print(f"✓ L'approche locale est plus efficace computationnellement (+{diff:.2f}x)")
        else:
            diff = safe_get(results_global, 'computational_efficiency') - safe_get(results_local, 'computational_efficiency')
            print(f"✗ L'approche locale est moins efficace computationnellement (-{diff:.2f}x)")
    else:
        print("⚠ Impossible de comparer l'efficacité computationnelle (valeurs non valides ou nulles)")

    # Temps d'exécution
    time_result = safe_compare(
        safe_get(results_local, 'total_time'),
        safe_get(results_global, 'total_time'),
        False
    )
    if time_result is not None:
        if time_result:
            diff = safe_get(results_global, 'total_time') - safe_get(results_local, 'total_time')
            print(f"✓ L'approche locale est plus rapide (-{diff:.2f} secondes)")
        else:
            diff = safe_get(results_local, 'total_time') - safe_get(results_global, 'total_time')
            print(f"✗ L'approche locale est plus lente (+{diff:.2f} secondes)")
    else:
        print("⚠ Impossible de comparer les temps d'exécution (valeurs non valides ou nulles)")

    print("-" * 80)

    # Conclusion
    advantages_local = 0
    advantages_analyzed = 0

    # Compression (plus élevé = mieux)
    comp_result = safe_compare(
        safe_get(results_local, 'compression_ratio'),
        safe_get(results_global, 'compression_ratio'),
        True
    )
    if comp_result is not None:
        advantages_analyzed += 1
        if comp_result:
            advantages_local += 1

    # Performance loss (plus bas = mieux)
    perf_result = safe_compare(
        safe_get(results_local, 'performance_loss'),
        safe_get(results_global, 'performance_loss'),
        False
    )
    if perf_result is not None:
        advantages_analyzed += 1
        if perf_result:
            advantages_local += 1

    # Recovery (plus élevé = mieux)
    recovery_result = safe_compare(
        safe_get(results_local, 'performance_recovery'),
        safe_get(results_global, 'performance_recovery'),
        True
    )
    if recovery_result is not None:
        advantages_analyzed += 1
        if recovery_result:
            advantages_local += 1

    # Efficiency (plus élevé = mieux)
    efficiency_result = safe_compare(
        safe_get(results_local, 'computational_efficiency'),
        safe_get(results_global, 'computational_efficiency'),
        True
    )
    if efficiency_result is not None:
        advantages_analyzed += 1
        if efficiency_result:
            advantages_local += 1

    # Time (plus bas = mieux)
    time_result = safe_compare(
        safe_get(results_local, 'total_time'),
        safe_get(results_global, 'total_time'),
        False
    )
    if time_result is not None:
        advantages_analyzed += 1
        if time_result:
            advantages_local += 1

    print("\nCONCLUSION :")
    if advantages_analyzed == 0:
        print("⚠ Impossible de conclure (données insuffisantes pour la comparaison)")
    elif advantages_local > advantages_analyzed / 2:
        print(f"✅ L'approche de clusterisation locale est globalement meilleure ({advantages_local}/{advantages_analyzed} avantages)")
    else:
        print(f"❌ L'approche de clusterisation globale reste préférable ({advantages_analyzed-advantages_local}/{advantages_analyzed} avantages)")

#if __name__ == "__main__":
    # Exécuter la comparaison des approches
   # results_global, results_local = main()
  #  print("\n=== Comparaison des approches terminée avec succès ===")

def main_two_phase():
    """
    Fonction principale pour l'approche en deux phases.
    """
    print("=" * 80)
    print("EXPÉRIENCE FEDERATED LEARNING AVEC APPROCHE EN DEUX PHASES")
    print("=" * 80)

    # Paramètres
    params = {
        "n_clients": 3,            # Nombre de clients
        "n_epochs": 50,             # Nombre d'époques pour l'entraînement local
        "n_clusters": 3,           # Nombre de clusters pour la compression
        "n_a": 32,                 # Dimension cachée
        "n_x": 8,                  # Dimension d'entrée
        "n_y": 5,                  # Nombre de classes
        "batch_size": 32,          # Taille du batch
        "sequence_length": 10,     # Longueur de séquence
        "num_transfer_epochs": 3,  # Époques pour le transfer learning
        "learning_rate": 0.01,     # Taux d'apprentissage
        "n_communication_rounds": 3, # Nombre de cycles de communication
        "use_synthetic_data": True  # Utiliser des données synthétiques
    }


     # Exécuter l'expérience
    results, final_model_proposed, final_model_fedavg = federated_main_two_phase_local(**params)


    # Afficher un tableau récapitulatif des performances
    print("\nRÉCAPITULATIF DES PERFORMANCES :")
    print("-" * 100)
    print("| Cycle | Proposée Acc | FedAvg Acc | Diff Acc (%) | Proposée Loss | FedAvg Loss | Diff Loss (%) |")
    print("-" * 100)

    for result in results:
        cycle = result["cycle"] + 1
        prop_acc = result["proposed_accuracy"]
        fedavg_acc = result["fedavg_accuracy"]
        prop_loss = result["proposed_loss"]
        fedavg_loss = result["fedavg_loss"]

        # Calcul des différences en pourcentage
        if fedavg_acc > 0:
            acc_diff = ((prop_acc - fedavg_acc) / fedavg_acc) * 100
        else:
            acc_diff = float('inf')

        if fedavg_loss > 0:
            loss_diff = ((prop_loss - fedavg_loss) / fedavg_loss) * 100
        else:
            loss_diff = float('inf')

        # Signe pour indiquer si c'est mieux (+) ou moins bien (-)
        acc_sign = "+" if acc_diff > 0 else ""
        loss_sign = "+" if loss_diff > 0 else ""

        print(f"| {cycle:5d} | {prop_acc:11.4f} | {fedavg_acc:9.4f} | {acc_sign}{acc_diff:10.2f} | {prop_loss:12.4f} | {fedavg_loss:10.4f} | {loss_sign}{loss_diff:11.2f} |")

    print("-" * 100)

    # Afficher un récapitulatif des économies de bande passante
    print("\nRÉCAPITULATIF DES ÉCONOMIES DE BANDE PASSANTE :")
    print("-" * 80)
    print("| Cycle | Traditionnelle (KB) | Proposée (KB) | Économie (%) |")
    print("-" * 80)

    total_trad = 0
    total_prop = 0

    for result in results:
        cycle = result["cycle"] + 1
        trad_kb = result["traditional_bandwidth"] / 1024
        prop_kb = result["proposed_bandwidth"] / 1024

        if cycle > 1:  # Phase 2
            saving = (1 - prop_kb/trad_kb) * 100
            print(f"| {cycle:5d} | {trad_kb:18.2f} | {prop_kb:13.2f} | {saving:11.2f} |")
        else:  # Phase 1
            print(f"| {cycle:5d} | {trad_kb:18.2f} | {prop_kb:13.2f} | {'N/A':11s} |")

        total_trad += result["traditional_bandwidth"]
        total_prop += result["proposed_bandwidth"]

    print("-" * 80)

    # Économie totale
    total_trad_kb = total_trad / 1024
    total_prop_kb = total_prop / 1024
    total_saving = (1 - total_prop/total_trad) * 100

    print(f"| Total | {total_trad_kb:18.2f} | {total_prop_kb:13.2f} | {total_saving:11.2f} |")
    print("-" * 80)

    # Conclusion
    print("\nCONCLUSION:")
    avg_acc_diff = sum([(r["proposed_accuracy"] - r["fedavg_accuracy"]) / r["fedavg_accuracy"] * 100 if r["fedavg_accuracy"] > 0 else 0 for r in results]) / len(results)
    avg_loss_diff = sum([(r["proposed_loss"] - r["fedavg_loss"]) / r["fedavg_loss"] * 100 if r["fedavg_loss"] > 0 else 0 for r in results]) / len(results)

    print(f"Différence moyenne de précision: {avg_acc_diff:.2f}%")
    print(f"Différence moyenne de perte: {avg_loss_diff:.2f}%")
    print(f"Économie moyenne de bande passante (cycles 2-3): {total_saving:.2f}%")

    if avg_acc_diff > -1 and total_saving > 50:  # Seuils arbitraires pour la conclusion
        print("VERDICT: La méthode proposée permet d'économiser significativement de la bande passante tout en maintenant des performances comparables à FedAvg.")
    elif avg_acc_diff < -5:
        print("VERDICT: La méthode proposée économise de la bande passante mais au détriment d'une baisse notable des performances.")
    else:
        print("VERDICT: Compromis modéré entre économie de bande passante et performances.")

    return results, final_model_proposed, final_model_fedavg


if __name__ == "__main__":
    main_two_phase()

EXPÉRIENCE FEDERATED LEARNING AVEC APPROCHE EN DEUX PHASES
=== Lancement du Federated Learning en deux phases avec clusterisation locale ===
Génération des données synthétiques...

=== Cycle de communication 1/3 ===
Phase 1: Initialisation avec FedAvg et création des centres de clusters
Entraînement du client 1/3
Époque 1/50
Loss: 1.6304
Époque 2/50
Loss: 1.6121
Époque 3/50
Loss: 1.5962
Époque 4/50
Loss: 1.5820
Époque 5/50
Loss: 1.5691
Époque 6/50
Loss: 1.5570
Époque 7/50
Loss: 1.5455
Époque 8/50
Loss: 1.5343
Époque 9/50
Loss: 1.5230
Époque 10/50
Loss: 1.5113
Époque 11/50
Loss: 1.4991
Époque 12/50
Loss: 1.4860
Époque 13/50
Loss: 1.4721
Époque 14/50
Loss: 1.4569
Époque 15/50
Loss: 1.4403
Époque 16/50
Loss: 1.4224
Époque 17/50
Loss: 1.4033
Époque 18/50
Loss: 1.3832
Époque 19/50
Loss: 1.3620
Époque 20/50
Loss: 1.3397
Époque 21/50
Loss: 1.3161
Époque 22/50
Loss: 1.2912
Époque 23/50
Loss: 1.2652
Époque 24/50
Loss: 1.2380
Époque 25/50
Loss: 1.2097
Époque 26/50
Loss: 1.1804
Époque 27/50
Loss:

C:\Users\ikram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] Le fichier spécifié est introuvable
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\ikram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fil

Résultats du cycle 1:
  Méthode proposée - Accuracy: 0.4389, Loss: 0.1926
  FedAvg traditionnel - Accuracy: 0.4389, Loss: 0.1926
  Bande passante traditionnelle: 63.43 KB
  Bande passante proposée: 253.84 KB

=== Cycle de communication 2/3 ===
Phase 2: Mise à jour avec matrices de transition uniquement
Client 1/3
Époque 1/50
Loss: 1.5322
Époque 2/50
Loss: 1.3779
Époque 3/50
Loss: 1.2809
Époque 4/50
Loss: 1.2111
Époque 5/50
Loss: 1.1562
Époque 6/50
Loss: 1.1098
Époque 7/50
Loss: 1.0672
Époque 8/50
Loss: 1.0259
Époque 9/50
Loss: 0.9855
Époque 10/50
Loss: 0.9469
Époque 11/50
Loss: 0.9108
Époque 12/50
Loss: 0.8773
Époque 13/50
Loss: 0.8456
Époque 14/50
Loss: 0.8151
Époque 15/50
Loss: 0.7854
Époque 16/50
Loss: 0.7565
Époque 17/50
Loss: 0.7280
Époque 18/50
Loss: 0.6997
Époque 19/50
Loss: 0.6719
Époque 20/50
Loss: 0.6451
Époque 21/50
Loss: 0.6196
Époque 22/50
Loss: 0.5951
Époque 23/50
Loss: 0.5713
Époque 24/50
Loss: 0.5484
Époque 25/50
Loss: 0.5262
Époque 26/50
Loss: 0.5047
Époque 27/50
Loss:

KeyboardInterrupt: 